# Project: prediction of BSI using machine learning based on biochemical variables


## Load data and libraries and set parameters

In [ ]:
import numpy as np
import pandas as pd
import sklearn
from sklearn.model_selection import train_test_split, RandomizedSearchCV, StratifiedKFold
from sklearn import preprocessing
import matplotlib.pyplot as plt
from sklearn import ensemble
import seaborn as sns
import os
%matplotlib inline

# Constants
SEED = 123
FOLDS = 5
VERBOSE = 0

# processing
n_cpu_for_tuning = 20
n_cpu_model_training = 40
n_iter = 3 # repeated cross validation for hyperparameter tunning
scale_data = False
data_complete = False # filter out incomplete samples (more than 50% of biochemical variables missing) - optional
from sklearn.metrics import make_scorer, matthews_corrcoef, balanced_accuracy_score, f1_score
tun_score = "roc_auc" # f1_score 
verbosity_option = -1 # no training details required to show
filter_highly_mis_feats = True # remove highly missing features (optional)
feat_sel = True # optional

# optimization
no_biochemical_variables = False
only_biochemical_variables = False
use_default_threshold = True
outcome_var = "BSIClass"
# lookback_period = 30 # optional
# gp_value = 0 # optional

ADMper= 7
devset_name = f'R/train_set_ADMper_{ADMper}'
testset_name = f'R/test_set_ADMper_{ADMper}'

devset = pd.read_csv(f'{devset_name}.csv')
testset = pd.read_csv(f'{testset_name}.csv')

main_folder_name = 'results'
subfolder_name = f'{devset_name}'

# Check if the main folder exists, and create it if not
if not os.path.exists(main_folder_name):
    os.makedirs(main_folder_name)

# Change the current working directory to the main folder
os.chdir(main_folder_name)

# Check if the subfolder exists, and create it if not
if not os.path.exists(subfolder_name):
    os.makedirs(subfolder_name)

# Change the current working directory to the subfolder
os.chdir(subfolder_name)


In [ ]:
import pandas as pd

category_counts = devset["Sex"].value_counts()
total_count = len(devset["Sex"])

# Number and percent of each category
result = pd.DataFrame({
    'Category': category_counts.index,
    'Count': category_counts.values,
    'Percent': (category_counts / total_count) * 100
})

# Display the result
print(result)


In [ ]:
import pandas as pd

category_counts = testset["Sex"].value_counts()
total_count = len(testset["Sex"])

# Number and percent of each category
result = pd.DataFrame({
    'Category': category_counts.index,
    'Count': category_counts.values,
    'Percent': (category_counts / total_count) * 100
})

# Display the result
print(result)


In [ ]:
if data_complete:
    df = devset.copy()

    # List of columns to exclude from the threshold check
    exclude_columns = ["ID","date","Sex","BSIClass","Age","allbac"]
    
    # Calculate the threshold for the remaining columns
    included_columns = [col for col in df.columns if col not in exclude_columns]
    threshold = int(0.5 * len(included_columns))
    
    # Remove rows with more than 50% missing values, considering only the included columns
    df_cleaned = df.dropna(subset=included_columns, thresh=threshold)
    
    print(df.shape)
    print(df_cleaned.shape)
    print(len(df_cleaned["ID"].unique()))
    del devset
    devset = df_cleaned.copy()


In [ ]:
if data_complete:
    df = testset.copy()
    
    # List of columns to exclude from the threshold check
    exclude_columns = ["ID","date","Sex","BSIClass","Age","allbac"]
    
    # Calculate the threshold for the remaining columns
    included_columns = [col for col in df.columns if col not in exclude_columns]
    threshold = int(0.5 * len(included_columns))
    
    # Remove rows with more than 50% missing values, considering only the included columns
    df_cleaned = df.dropna(subset=included_columns, thresh=threshold)
    
    print(df.shape)
    print(df_cleaned.shape)
    print(len(df_cleaned["ID"].unique()))
    del testset
    testset = df_cleaned.copy()

In [ ]:
devset = devset.reset_index(drop=True)
testset = testset.reset_index(drop=True)

In [ ]:
len(devset["ID"].unique())

In [ ]:
len(testset["ID"].unique())

In [ ]:
testset.columns

In [ ]:
# Save back up data
devset_backup = devset.copy()
testset_backup = testset.copy()

# Drop columns from the original datasets
devset = devset.drop(columns=['ID', 'date', 'allbac'])
testset = testset.drop(columns=['ID', 'date', 'allbac'])


In [ ]:
devset_withmissing = devset.copy()
testset_withmissing = testset.copy()

In [ ]:
if no_biochemical_variables:
    non_biochemical_cols_to_use = ['Sex', 'Age', outcome_var]
    devset_withmissing = devset_withmissing[non_biochemical_cols_to_use]
    testset_withmissing = testset_withmissing[non_biochemical_cols_to_use]


In [ ]:
if only_biochemical_variables:
    non_biochemical_cols_to_use = ['Sex', 'Age']
    devset_withmissing = devset_withmissing.drop(columns=non_biochemical_cols_to_use)
    testset_withmissing = testset_withmissing.drop(columns=non_biochemical_cols_to_use)


In [ ]:
testset_withmissing.shape

In [ ]:
devset_withmissing.columns

In [ ]:
if only_biochemical_variables==False:
    common_sex_category = 'Male'

    devset_withmissing['Sex'] = pd.Categorical(devset_withmissing['Sex'], categories=[common_sex_category])
    devset_withmissing = pd.get_dummies(devset_withmissing, columns=['Sex'], prefix='Sex')

    testset_withmissing['Sex'] = pd.Categorical(testset_withmissing['Sex'], categories=[common_sex_category])
    testset_withmissing = pd.get_dummies(testset_withmissing, columns=['Sex'], prefix='Sex')


In [ ]:
mydata = pd.concat([devset_withmissing, testset_withmissing])
mydata.columns

In [ ]:
mydata.dtypes

##### limit the number of lines etc. to display

In [ ]:
pd.set_option('display.max_rows', 50)
pd.set_option('display.max_columns', 100)
pd.set_option('display.width', 100)

### data overview

#### display the type of the variables (columns)

#### filter out the data based on observation period

In [ ]:
mydata.shape

#### check what columns have missing values

In [ ]:
# Identify columns with missing values
columns_with_missing_values = mydata.columns[mydata.isnull().any()]
columns_with_missing_values

In [ ]:
mydata[outcome_var].unique()

In [ ]:
testset.describe()

In [ ]:
devset.describe()

In [ ]:
mydata.describe()

In [ ]:
mydata

#### check for missingness

In [ ]:
# import numpy as np

# Step 1: Calculate the total number of missing values for each column
missing_values = mydata.isnull().sum()

# Step 2: Divide by the total number of rows
total_rows = len(mydata)
missing_percentage = (missing_values / total_rows) * 100

# Step 3: Round the percentages to two decimal points
missing_percentage = missing_percentage.round(2)

# Step 4: Sort the percentages in ascending order
missing_percentage = missing_percentage.sort_values(ascending=False)

# Step 5: Calculate the mean and standard deviation of the missingness
mean_missingness = np.mean(missing_percentage)
std_missingness = np.std(missing_percentage)

# Step 6: Display the missing percentages, mean, and standard deviation
print("Missing Value Percentages:")
print(missing_percentage)
print("Mean ± Standard Deviation of Missingness: {:.2f} ± {:.2f}".format(mean_missingness, std_missingness))


In [ ]:
devset_withmissing.dtypes

In [ ]:
import pandas as pd
import numpy as np
from scipy.stats import pointbiserialr
from joblib import Parallel, delayed

df_imputed = devset_withmissing.copy()

# Convert 'outcome_var' to numerical variable
df_imputed[outcome_var] = pd.factorize(df_imputed[outcome_var])[0]

# Calculate point-biserial correlation for each feature multiple times
num_iterations = 1000  # You can adjust the number of iterations as needed
biserial_corr_values = []

def calculate_biserial_corr(iteration, df, outcome_var):
    # Impute NaNs with random values within each iteration
    df_copy = df.copy()
    for col in df_copy.columns:
        col_missing = df_copy[col].isnull()
        if col_missing.sum() > 0:
            random_values = np.random.choice(df_copy.loc[~col_missing, col], size=col_missing.sum())
            df_copy.loc[col_missing, col] = random_values
    
    biserial_corr_iter = [pointbiserialr(df_copy[col], df_copy[outcome_var])[0] for col in df_copy.drop(outcome_var, axis=1)]
    return biserial_corr_iter

# Calculate point-biserial correlation for each feature multiple times
biserial_corr_values = Parallel(n_jobs=50)(delayed(calculate_biserial_corr)(i, df_imputed, outcome_var) for i in range(num_iterations))

# Calculate the lower and upper quantiles of point-biserial correlation for each feature
lower_quantile_corr = np.percentile(biserial_corr_values, 25, axis=0)
median_corr = np.percentile(biserial_corr_values, 50, axis=0)
upper_quantile_corr = np.percentile(biserial_corr_values, 75, axis=0)

# Create a DataFrame with feature names, lower and upper quantiles of point-biserial correlation
corr_summary_df = pd.DataFrame({'Feature': df_imputed.drop(outcome_var, axis=1).columns,
                                'median_Corr': median_corr,
                                'Lower_Quantile_Corr': lower_quantile_corr,
                                'Upper_Quantile_Corr': upper_quantile_corr})
corr_summary_df

In [ ]:
corr_summary_df

In [ ]:
import pandas as pd
import numpy as np
from scipy.stats import pointbiserialr, wilcoxon
from joblib import Parallel, delayed

df_imputed = devset_withmissing.copy()

# Convert 'outcome_var' to numerical variable
df_imputed[outcome_var] = pd.factorize(df_imputed[outcome_var])[0]

# Calculate point-biserial correlation for each feature multiple times
num_iterations = 1000  # You can adjust the number of iterations as needed
biserial_corr_values = []

def calculate_biserial_corr(iteration, df, outcome_var):
    df_copy = df.copy()
    for col in df_copy.columns:
        col_missing = df_copy[col].isnull()
        if col_missing.sum() > 0:
            random_values = np.random.choice(df_copy.loc[~col_missing, col], size=col_missing.sum())
            df_copy.loc[col_missing, col] = random_values
    
    biserial_corr_iter = [pointbiserialr(df_copy[col], df_copy[outcome_var])[0] for col in df_copy.drop(outcome_var, axis=1)]
    return biserial_corr_iter

# Calculate point-biserial correlation for each feature multiple times
biserial_corr_values = Parallel(n_jobs=50)(delayed(calculate_biserial_corr)(i, df_imputed, outcome_var) for i in range(num_iterations))

# Create a DataFrame with feature names and correlation coefficients from all iterations
corr_df = pd.DataFrame(biserial_corr_values, columns=df_imputed.drop(outcome_var, axis=1).columns)


In [ ]:
from scipy.stats import pointbiserialr, norm, iqr
# Calculate the lower and upper quantiles of point-biserial correlation for each feature
lower_quantile_corr = np.percentile(corr_df, 25, axis=0)
median_corr = np.percentile(corr_df, 50, axis=0)
upper_quantile_corr = np.percentile(corr_df, 75, axis=0)

# Filter features based on quantiles
significant_features = [feature for feature, lower, upper in zip(corr_df.columns, lower_quantile_corr, upper_quantile_corr) if lower > 0 or upper < 0]

# Filter the original DataFrame to include only significant features
df_imputed_filtered = df_imputed[significant_features]


In [ ]:
# Mark significant features with a different color
corr_summary_df['Color'] = np.where(corr_summary_df['Feature'].isin(significant_features), 'significant', 'not significant')

# Sort DataFrame by median correlation for better visualization
corr_summary_df = corr_summary_df.sort_values(by='median_Corr', ascending=False)

# Plotting
plt.figure(figsize=(10, 6))

# Plot significant features in red and non-significant features in blue
sns.pointplot(x='median_Corr', y='Feature', data=corr_summary_df, hue='Color', dodge=True, join=False)

# Plot error bars for all features
plt.errorbar(x=corr_summary_df['median_Corr'], y=corr_summary_df['Feature'],
             xerr=[corr_summary_df['median_Corr'] - corr_summary_df['Lower_Quantile_Corr'], corr_summary_df['Upper_Quantile_Corr'] - corr_summary_df['median_Corr']],
             fmt='o', color='black', capsize=5, label='IQR')

# Customize plot
plt.title('median and quantile correlation coefficients for features')
plt.xlabel('PB correlation values')
plt.ylabel('Feature')
plt.legend()
plt.grid(True)
plt.show()

In [ ]:
significant_features

In [ ]:
import pandas as pd
import numpy as np
from sklearn.feature_selection import mutual_info_classif
from joblib import Parallel, delayed

# outcome_var = 'outcome_var'
df_imputed = devset_withmissing.copy()

# Add a random feature with missing values to df_imputed
df_imputed['random_feature'] = np.random.choice([np.nan, np.random.rand()], size=len(df_imputed))

# Calculate mutual information for each feature multiple times, including the random feature
num_iterations = 1000  # You can adjust the number of iterations as needed
mutual_info_values = []

def calculate_mutual_info(iteration, df, outcome_var):
    # Impute NaNs with random values within each iteration
    df_copy = df.copy()
    for col in df_copy.columns:
        col_missing = df_copy[col].isnull()
        if col_missing.sum() > 0:
            random_values = np.random.choice(df_copy.loc[~col_missing, col], size=col_missing.sum())
            df_copy.loc[col_missing, col] = random_values
    
    mutual_info_iter = mutual_info_classif(df_copy.drop(outcome_var, axis=1), df_copy[outcome_var])
    return mutual_info_iter

# Calculate mutual information for each feature multiple times, including the random feature
mutual_info_values = Parallel(n_jobs=50)(delayed(calculate_mutual_info)(i, df_imputed, outcome_var) for i in range(num_iterations))

# Calculate the lower and upper quantiles of mutual information for each feature
lower_quantile_mi = np.percentile(mutual_info_values, 25, axis=0)
median_mi = np.percentile(mutual_info_values, 50, axis=0)
upper_quantile_mi = np.percentile(mutual_info_values, 75, axis=0)

# Create a DataFrame with feature names, lower and upper quantiles of mutual information
mi_summary_df = pd.DataFrame({'Feature': df_imputed.drop(outcome_var, axis=1).columns,'median_MI':median_mi, 'Lower_Quantile_MI': lower_quantile_mi, 'Upper_Quantile_MI': upper_quantile_mi})

# Get median_random_feature_mi from the mutual_info_values
median_random_feature_mi = np.median(mutual_info_values, axis=0)

# Filter out 


In [ ]:
import matplotlib.pyplot as plt

# Sort mi_summary_df by Median_MI
mi_summary_df_sorted = mi_summary_df.sort_values(by='median_MI', ascending=False)

In [ ]:
mi_summary_df_sorted

In [ ]:
df = mi_summary_df_sorted.copy()


# Plotting
plt.figure(figsize=(10, 6))

# Swap 'Feature' and 'median_MI' in the plot functions
sns.pointplot(x='median_MI', y='Feature', data=df, color='blue', label='Median MI')

# Swap 'Feature' and 'median_MI' in the errorbar function
plt.errorbar(x=df['median_MI'], y=df['Feature'], xerr=[df['median_MI'] - df['Lower_Quantile_MI'], df['Upper_Quantile_MI'] - df['median_MI']],
             fmt='o', color='blue', capsize=5, label='Quantile MI Range')

# Set smaller font size
sns.set(font_scale=0.6)
# Customize plot
plt.title('Median and Quantile MI for Features')
plt.xlabel('MI Value')
plt.ylabel('Feature')
plt.legend()
plt.grid(True)
plt.show()

In [ ]:
df1 = pd.DataFrame(devset_withmissing)
df2 = pd.DataFrame(testset_withmissing)
# Function to create the summary table for a single dataset
def create_summary_table(dataframe, dataset_name):
    summary_data = {'Variable': [], 'Value': []}

    for col in sorted(dataframe.columns):  # Sort variable names alphabetically
        # Variable Name
        summary_data['Variable'].append(col)
        summary_data['Value'].append('Variable Name')

        # For numerical variables - Median (lower quantile, higher quantile)
        if pd.api.types.is_numeric_dtype(dataframe[col]):
            median = dataframe[col].median()
            q25 = dataframe[col].quantile(0.25)
            q75 = dataframe[col].quantile(0.75)
            summary_data['Variable'].append('')
            summary_data['Value'].append(f'{median:.2f} ({q25:.2f}, {q75:.2f})')

        # For categorical variables - Categories, Counts, and Percentages
        elif pd.api.types.is_categorical_dtype(dataframe[col]):
            categories = dataframe[col].value_counts()
            total_count = len(dataframe[col])
            summary_data['Variable'].extend(['', ''])
            summary_data['Value'].extend(['Categories', 'Counts'])
            
            for category, count in categories.items():
                percentage = (count / total_count) * 100
                summary_data['Variable'].append(category)
                summary_data['Value'].append(f'{count} - {percentage:.2f}%')

        # Missing values for all variable types
        missing_count = dataframe[col].isnull().sum()
        missing_percentage = (missing_count / len(dataframe)) * 100
        summary_data['Variable'].append('')
        summary_data['Value'].append(f' {missing_percentage:.2f}%')

    # Add a column for the dataset name
    summary_data['Variable'].append('Dataset')
    summary_data['Value'].append(dataset_name)

    summary_df = pd.DataFrame(summary_data)
    return summary_df

# Create summary tables for each dataset
summary_table1 = create_summary_table(df1, 'dataset1')
summary_table2 = create_summary_table(df2, 'dataset2')

# Concatenate summary tables horizontally
final_summary_table = pd.concat([summary_table1.set_index('Variable'), summary_table2.set_index('Variable')], axis=1).reset_index()

# Display the final summary table
print(final_summary_table)


In [ ]:
# Save the final summary table to an Excel file
final_summary_table.to_excel('summary_table.xlsx', index=False)


In [ ]:
if feat_sel: # feature selection
    devset_withmissing = devset_withmissing[significant_features + [outcome_var]]
    testset_withmissing = testset_withmissing[significant_features + [outcome_var]]


In [ ]:
# Specify the numerical features you want to scale
numerical_columns = devset_withmissing.select_dtypes(include=['float64', 'int64']).columns
has_negative_values = np.any(devset_withmissing[numerical_columns] < 0)

if has_negative_values:
    print("There are negative values in devset_withmissing.")
else:
    print("There are no negative values in devset_withmissing.")

# Display the minimum value of each column
min_values = devset_withmissing[numerical_columns].min()
print("\nMinimum values of each column:")
print(min_values)


In [ ]:
# import numpy as np

# Step 1: Calculate the total number of missing values for each column
missing_values = devset_withmissing.isnull().sum()

# Step 2: Divide by the total number of rows
total_rows = len(devset_withmissing)
missing_percentage = (missing_values / total_rows) * 100

# Step 3: Round the percentages to two decimal points
missing_percentage = missing_percentage.round(2)

# Step 4: Sort the percentages in ascending order
missing_percentage = missing_percentage.sort_values(ascending=False)

# Step 5: Calculate the mean and standard deviation of the missingness
mean_missingness = np.mean(missing_percentage)
std_missingness = np.std(missing_percentage)

# Step 6: Display the missing percentages, mean, and standard deviation
print("Missing Value Percentages:")
print(missing_percentage)
print("Mean ± Standard Deviation of Missingness: {:.2f} ± {:.2f}".format(mean_missingness, std_missingness))


In [ ]:
# Step 7: Identify columns with more than 90% missingness
highly_missing_features = missing_percentage[missing_percentage > 90].index.tolist()

# Step 8: Print the highly missing features
print("Columns with more than 90% missingness:")
print(highly_missing_features)

if filter_highly_mis_feats:
    # Step 9: Drop columns with more than 90% missingness from devset_withmissing
    devset_withmissing = devset_withmissing.drop(columns=highly_missing_features)
    
    # Step 10: Drop columns with more than 90% missingness from testset_withmissing
    testset_withmissing = testset_withmissing.drop(columns=highly_missing_features)


In [ ]:
from sklearn.preprocessing import RobustScaler # optional
if scale_data:
    # Specify the numerical features you want to scale
    numerical_columns = devset_withmissing.select_dtypes(include=['float64', 'int64']).columns
    
    robust_scaler = RobustScaler().fit(devset_withmissing[numerical_columns])
    
    # Use the RobustScaler to scale the numerical features
    devset_withmissing[numerical_columns] = robust_scaler.fit_transform(devset_withmissing[numerical_columns])
    testset_withmissing[numerical_columns] = robust_scaler.fit_transform(testset_withmissing[numerical_columns])

In [ ]:
numerical_columns

In [ ]:
import pandas as pd
import numpy as np
from sklearn.experimental import enable_iterative_imputer
from sklearn.impute import IterativeImputer

# Separate features and target variable for the training set
X_train_withmissing = devset_withmissing.drop(outcome_var, axis=1)
y_train = devset_withmissing[outcome_var]

# Separate features and target variable for the test set
X_test_withmissing = testset_withmissing.drop(outcome_var, axis=1)
y_test = testset_withmissing[outcome_var]

min_values = X_train_withmissing.min()
max_values = X_train_withmissing.max()

if scale_data:
    mice_imputer = IterativeImputer(
        max_iter=10, 
        random_state=SEED, tol=1e-4
    )
else:
    mice_imputer = IterativeImputer(max_iter=10,  random_state=SEED, min_value=min_values, max_value=max_values, tol=1e-4)
# Fit and transform X_train_withmissing with MICE imputation
X_train_imputed = mice_imputer.fit_transform(X_train_withmissing)

# Convert the imputed array back to a DataFrame with column names
X_train_imputed = pd.DataFrame(X_train_imputed, columns=X_train_withmissing.columns)

# Combine X_train_imputed and y_train into a single DataFrame
devset_imputed = pd.concat([X_train_imputed, y_train], axis=1)

# Transform X_test_withmissing using the same MICE imputer
X_test_imputed = mice_imputer.transform(X_test_withmissing)

# Convert the imputed array back to a DataFrame with column names
X_test_imputed = pd.DataFrame(X_test_imputed, columns=X_test_withmissing.columns)

# Combine X_test_imputed and y_test into a single DataFrame
testset_imputed = pd.concat([X_test_imputed, y_test], axis=1)


#### check the number of rows and columns of the dataset

In [ ]:
mydata.shape

#### Use data dictionary to have a description of the variables in results

using the data dictionary the name of the variables are displayed in figures and tables in publishable form as in a research article for example

In [ ]:
data_dictionary = {'Sodium': 'Sodium', 'ALAT': 'Alanine Aminotransferase', 'ALB': 'Albumin', 'APTT': 'Activated Partial Thromboplastin Time',
       'BASO': 'Basophils', 'CA': 'Calcium', 'CAI': 'Calcium Ionized', 'CHOL': 'Cholesterol', 'CL': 'Chloride',
       'CREA': 'Creatinine', 'CRP': 'C-Reactive Protein', 'EOS': 'Eosinophils', 'ERY': 'Erythrocytes',
       'FERRIT': 'Ferritin', 'GGT': 'Gamma-glutamyltransferase', 'GLU': 'Glucose', 'GLUF': 'Glycated Hemoglobin',
       'HAPTO': 'Haptoglobin', 'HB': 'Hemoglobin', 'HBA1C': 'Hemoglobin A1c', 'HDL': 'High-density lipoprotein',
       'INR': 'International Normalized Ratio', 'JERN': 'Iron', 'K': 'Potassium', 'LDH': 'Lactate dehydrogenase',
       'LDL': 'Low-density lipoprotein', 'LEU': 'Leukocytes', 'LYMFO': 'Lymphocytes', 'MG': 'Magnesium',
       'MONO': 'Monocytes', 'NEUTRO': 'Neutrophils', 'PROCAL': 'Procalcitonin', 'THROM': 'Platelets',
       'TRANS': 'Transferrin', 'TRIG': 'Triglycerides', 'TSH': 'Thyroid-stimulating hormone',
       'Sex': 'Sex', 'Age': 'Age', 'BSIClass': 'BSI Class', 'PrevAdmissionRate': 'Previous Medical Encounter Rate',
       'PrevInfectionRate': 'Previous Infection Rate', 'biochemical_abnormality_score': 'Biochemical Abnormality Score',
       'biochemical_abnormality_score_NAexcluded': 'Adjusted Biochemical Abnormality Score',
       'modified_biochemical_abnormality_score': 'Minimal Biochemical Abnormality Score', 'Sodium_base': 'Sodium (Baseline)',
       'ALAT_base': 'Alanine Aminotransferase (Baseline)', 'ALB_base': 'Albumin (Baseline)', 'APTT_base': 'Activated Partial Thromboplastin Time (Baseline)',
       'BASO_base': 'Basophils (Baseline)', 'CA_base': 'Calcium (Baseline)', 'CAI_base': 'Calcium Ionized (Baseline)',
       'CHOL_base': 'Cholesterol (Baseline)', 'CL_base': 'Chloride (Baseline)', 'CREA_base': 'Creatinine (Baseline)',
       'CRP_base': 'C-Reactive Protein (Baseline)', 'EOS_base': 'Eosinophils (Baseline)', 'ERY_base': 'Erythrocytes (Baseline)',
       'FERRIT_base': 'Ferritin (Baseline)', 'GGT_base': 'Gamma-glutamyltransferase (Baseline)', 'GLU_base': 'Glucose (Baseline)',
       'GLUF_base': 'Glycated Hemoglobin (Baseline)', 'HAPTO_base': 'Haptoglobin (Baseline)', 'HB_base': 'Hemoglobin (Baseline)',
       'HBA1C_base': 'Hemoglobin A1c (Baseline)', 'HDL_base': 'High-density lipoprotein (Baseline)','INR_base': 'International Normalized Ratio (Baseline)',
       'JERN_base': 'Iron (Baseline)', 'K_base': 'Potassium (Baseline)', 'LDH_base': 'Lactate dehydrogenase (Baseline)',
       'LDL_base': 'Low-density lipoprotein (Baseline)', 'LEU_base': 'Leukocytes (Baseline)', 'LYMFO_base': 'Lymphocytes (Baseline)',
       'MG_base': 'Magnesium (Baseline)', 'MONO_base': 'Monocytes (Baseline)', 'NEUTRO_base': 'Neutrophils (Baseline)',
       'PROCAL_base': 'Procalcitonin (Baseline)', 'THROM_base': 'Platelets (Baseline)', 'TRANS_base': 'Transferrin (Baseline)',
       'TRIG_base': 'Triglycerides (Baseline)', 'TSH_base': 'Thyroid-stimulating hormone (Baseline)',
       "NEUTRO_to_LYMFO": "Neutrophils-to-lymphocytes ratio",
       "Platelet-to-lymphocyte": "Platelet-to-lymphocyte ratio",
}
       # "PBCR": "Extended positive blood culture rate",
       # "CER": "Clinical encounter rate",
       # "BVA": "Biochemical variable abnormality",
       # "NBVA": "Non-missing biochemical variable abnormality",
       # "SBVA": "Subset biochemical variable abnormality",
       # "modifiedPIR": "Positive blood culture rate"

#### check the outcome variable and its categories (binary)

In [ ]:
mydata[outcome_var].unique()

#### data visualization

an overview of the distribution of the variables (continuous and categorical) is provided to compare the distributions per class (outcome category)

In [ ]:
import matplotlib.pyplot as plt
import numpy as np

mydata[outcome_var] = mydata[outcome_var].map({"noBSI": 'negative', "BSI": 'positive'}).astype('category')


In [ ]:

mydata = pd.concat([devset_imputed, testset_imputed])


In [ ]:
devset_imputed[outcome_var].unique()

In [ ]:
testset_imputed[outcome_var].unique()

In [ ]:
devset_withmissing[outcome_var].unique()

In [ ]:
testset_withmissing[outcome_var].unique()

In [ ]:
mydata[outcome_var].unique()

In [ ]:
mydata[outcome_var].unique()

In [ ]:
mydata.dtypes

In [ ]:
mydata[outcome_var]

In [ ]:
mydata[outcome_var]

In [ ]:
import matplotlib.pyplot as plt
import numpy as np

continuous_vars = mydata.select_dtypes(include=['float64', 'int64'])
categorical_vars = mydata.select_dtypes(include=['category'])
outcome_variable = mydata[outcome_var].copy()

# Calculate the number of rows and columns for subplots
num_continuous_vars = len(continuous_vars.columns)
num_categorical_vars = len(categorical_vars.columns)
num_cols_to_plot = 3
num_rows = (num_continuous_vars + num_categorical_vars + num_cols_to_plot - 1) // num_cols_to_plot + 1  # Adjust the number of rows based on the number of variables

# Create subplots for continuous variables
fig, axes = plt.subplots(num_rows, num_cols_to_plot, figsize=(12, num_rows * 2))  # Adjust the figsize as desired

# Iterate over continuous variables
for i, column in enumerate(continuous_vars.columns):
    # Determine the subplot indices
    row_idx = i // num_cols_to_plot
    col_idx = i % num_cols_to_plot

    # Check if subplot index is within the bounds of axes
    if row_idx < num_rows:
        # Get the axis for the current subplot
        ax = axes[row_idx, col_idx]

        # Iterate over each outcome category
        for outcome_category, ax_offset in zip(outcome_variable.unique(), [-0.2, 0.2]):
            # Filter the data for the current outcome category
            filtered_data = continuous_vars[outcome_variable == outcome_category][column]

            # Create a box plot for the current outcome category
            positions = np.array([1 + ax_offset])
            ax.boxplot(filtered_data.dropna(), positions=positions, widths=0.3, vert=False)  # Vert=False for horizontal box plots

        ax.set_title(f'{column}', fontsize=8)
        ax.set_yticks([1 - ax_offset, 1 + ax_offset])
        ax.set_yticklabels(outcome_variable.unique(), fontsize=8)
        ax.tick_params(axis='both', labelsize=8)
        ax.legend(fontsize=6)

# Iterate over categorical variables
for i, column in enumerate(categorical_vars.columns):
    # Determine the subplot indices
    row_idx = (i + num_continuous_vars) // num_cols_to_plot
    col_idx = (i + num_continuous_vars) % num_cols_to_plot

    # Check if subplot index is within the bounds of axes
    if row_idx < num_rows:
        # Get the axis for the current subplot
        ax = axes[row_idx, col_idx]

        # Normalize the counts for the current categorical variable stratified by outcome variable
        category_counts = categorical_vars.groupby(outcome_variable)[column].value_counts(normalize=True).unstack()
        category_counts.plot(kind='barh', ax=ax)

        # Set the title with the feature name
        ax.set_title(f'{column}', fontsize=8)

        ax.set_ylabel(None)
        ax.tick_params(axis='both', labelsize=8)
        ax.legend(fontsize=6)

# Remove any empty subplots at the end
if num_continuous_vars + num_categorical_vars < num_rows * num_cols_to_plot:
    for i in range(num_continuous_vars + num_categorical_vars, num_rows * num_cols_to_plot):
        fig.delaxes(axes.flatten()[i])

# Remove the subplot for outcome_var at the end
if num_continuous_vars + num_categorical_vars == num_rows * num_cols_to_plot - 1:
    last_ax_index = num_continuous_vars + num_categorical_vars - 1
    if last_ax_index >= 0:
        fig.delaxes(axes.flatten()[last_ax_index])

# Adjust the layout and spacing
plt.tight_layout()

# Show the plot
plt.show()


##### check out the distribution of the samples per class in the training (development) set and test set

In [ ]:
# Count the number of samples per class in devset
devset_class_counts = devset_withmissing[outcome_var].value_counts()

# Calculate the percentage of samples per class in devset
devset_class_percentages = (devset_class_counts / len(devset_withmissing)) * 100

# Count the number of samples per class in testset
testset_class_counts = testset_withmissing[outcome_var].value_counts()

# Calculate the percentage of samples per class in testset
testset_class_percentages = (testset_class_counts / len(testset_withmissing)) * 100

# Display the summary of the number of samples per class and their percentages
print("Development Set:")
print(devset_class_counts)
print(devset_class_percentages)

print("\nTest Set:")
print(testset_class_counts)
print(testset_class_percentages)


In [ ]:
cat_features = [] # no categorical feature is declared

In [ ]:
devset_imputed.loc[:, outcome_var] = devset_imputed[outcome_var].replace({"BSI": True, "noBSI": False}).astype(bool)
testset_imputed.loc[:, outcome_var] = testset_imputed[outcome_var].replace({"BSI": True, "noBSI": False}).astype(bool)

##### function to evaluate models and generate ROC curve, PR curve and confusion matrix

In [ ]:
import matplotlib.pyplot as plt
from sklearn import metrics
from sklearn.metrics import confusion_matrix, ConfusionMatrixDisplay, precision_recall_curve, roc_auc_score, brier_score_loss
import numpy as np
import pandas as pd
from sklearn.ensemble import HistGradientBoostingClassifier
import catboost as cb
import lightgbm as lgb
from sklearn.ensemble import RandomForestClassifier
from imblearn.ensemble import BalancedRandomForestClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.naive_bayes import GaussianNB

# Define a function for bootstrap sampling
def bootstrap_sample(data, n_samples):
    indices = np.random.randint(0, len(data), (n_samples, len(data)))
    return data[indices]

def calculate_confidence_interval(metric_values, alpha=0.95):
    # Filter out NaN values from metric_values
    non_nan_values = metric_values[~np.isnan(metric_values)]
    
    # Check if there are non-NaN values to calculate the confidence interval
    if len(non_nan_values) == 0:
        return np.nan, np.nan
    
    # Calculate confidence intervals for non-NaN values
    lower_bound = np.percentile(non_nan_values, (1 - alpha) / 2 * 100)
    upper_bound = np.percentile(non_nan_values, (1 + alpha) / 2 * 100)
    return lower_bound, upper_bound

def evaluate_and_plot_model(model, testset, y_test, class_labels, filename, threshold=0.5, bootstrap_samples=1000, min_positive_instances=1):


    bootstrap_values = []

    for _ in range(bootstrap_samples):
        # Perform bootstrap sampling
        bootstrap_sample_indices = np.random.choice(len(testset), len(testset), replace=True)
        bootstrap_sample_testset = testset.iloc[bootstrap_sample_indices]
        bootstrap_sample_y_test = y_test.iloc[bootstrap_sample_indices]

        if isinstance(model, (cb.CatBoostClassifier, lgb.LGBMClassifier, GaussianNB,RandomForestClassifier, LogisticRegression, BalancedRandomForestClassifier, HistGradientBoostingClassifier)):
            predictions = model.predict_proba(bootstrap_sample_testset)[:, 1]
            # print(predictions)
        else:
            predictions = model.predict(bootstrap_sample_testset)

        predictions_class = [True if x >= threshold else False for x in predictions]

        # Check if the number of positive instances is below the threshold
        if np.sum(bootstrap_sample_y_test) < min_positive_instances:
            # Set metrics to NaN or another suitable value
            PPV, NPV, sensitivity, specificity, balanced_accuracy, MCC, roc_auc, pr_auc, brier_score = [np.nan] * 9
        else:
            cm = confusion_matrix(bootstrap_sample_y_test, predictions_class)
            tn, fp, fn, tp = cm.ravel()

            PPV = tp / (tp + fp)
            NPV = tn / (tn + fn)
            sensitivity = tp / (tp + fn)
            specificity = tn / (tn + fp)
            balanced_accuracy = (sensitivity + specificity) / 2
            MCC = (tp * tn - fp * fn) / ((tp + fp) * (tp + fn) * (tn + fp) * (tn + fn)) ** 0.5
            roc_auc = roc_auc_score(y_true=bootstrap_sample_y_test, y_score=predictions)
            brier_score = brier_score_loss(y_true=bootstrap_sample_y_test, y_prob=predictions, pos_label=True)
            precision, recall, _ = precision_recall_curve(y_true=bootstrap_sample_y_test, probas_pred=predictions, pos_label=True)
            pr_auc = metrics.auc(x=recall, y=precision)

        # Store the metric values for each bootstrap iteration
        bootstrap_values.append([PPV, NPV, sensitivity, specificity, balanced_accuracy, MCC, roc_auc, pr_auc, brier_score])


    # Convert the list of metric values into a numpy array for easier manipulation
    bootstrap_values = np.array(bootstrap_values)

    # Calculate confidence intervals for each metric
    lower_bounds, upper_bounds = zip(*[calculate_confidence_interval(bootstrap_values[:, i]) for i in range(bootstrap_values.shape[1])])

    # Calculate the measures for the whole testset
    if np.sum(y_test) < min_positive_instances:
        # Set metrics to NaN or another suitable value
        PPV, NPV, sensitivity, specificity, balanced_accuracy, MCC, roc_auc, pr_auc, brier_score = [np.nan] * 9
    else:
        if isinstance(model, (cb.CatBoostClassifier, lgb.LGBMClassifier, GaussianNB,RandomForestClassifier, LogisticRegression,BalancedRandomForestClassifier, HistGradientBoostingClassifier)):
            predictions = model.predict_proba(testset)[:, 1]
            # print(predictions)
        else:
            predictions = model.predict(testset)

        predictions_class = [True if x >= threshold else False for x in predictions]

        cm = confusion_matrix(y_test, predictions_class)
        tn, fp, fn, tp = cm.ravel()

        PPV = tp / (tp + fp)
        NPV = tn / (tn + fn)
        sensitivity = tp / (tp + fn)
        specificity = tn / (tn + fp)
        balanced_accuracy = (sensitivity + specificity) / 2
        MCC = (tp * tn - fp * fn) / ((tp + fp) * (tp + fn) * (tn + fp) * (tn + fn)) ** 0.5
        roc_auc = roc_auc_score(y_true=y_test, y_score=predictions)
        brier_score = brier_score_loss(y_true=y_test, y_prob=predictions, pos_label=True)
        precision, recall, _ = precision_recall_curve(y_true=y_test, probas_pred=predictions, pos_label=True)
        pr_auc = metrics.auc(x=recall, y=precision)

    # Convert the list of metric values into a numpy array for easier manipulation
    bootstrap_values = np.array(bootstrap_values)

    # Calculate confidence intervals for each metric
    lower_bounds, upper_bounds = zip(*[calculate_confidence_interval(bootstrap_values[:, i]) for i in range(bootstrap_values.shape[1])])

    # Calculate the measures for the whole testset
    results = {
        'Metric': ['PPV', 'NPV', 'Sensitivity', 'Specificity', 'Balanced Accuracy', 'MCC', 'ROCAUC', 'PRAUC', 'Brier Score'],
        'Value': [PPV, NPV, sensitivity, specificity, balanced_accuracy, MCC, roc_auc, pr_auc, brier_score],
        'Lower Bound': lower_bounds,
        'Upper Bound': upper_bounds
    }

    results_df = pd.DataFrame(results)
    results_df['Value'] = results_df['Value'].round(2)
    results_df['Lower Bound'] = results_df['Lower Bound'].round(2)
    results_df['Upper Bound'] = results_df['Upper Bound'].round(2)
    print(results_df)

    fpr, tpr, thresholds = metrics.roc_curve(y_test, predictions, pos_label=True, drop_intermediate=False)
    precision, recall, pr_thresholds = precision_recall_curve(y_test, predictions, pos_label=True)

    # Finding the index closest to the custom threshold instead of 0.5
    threshold_index = (np.abs(thresholds - threshold)).argmin()
    threshold_custom = thresholds[threshold_index]
    tpr_custom = tpr[threshold_index]
    fpr_custom = fpr[threshold_index]

    pr_threshold_index = (np.abs(pr_thresholds - threshold)).argmin()
    pr_threshold_custom = pr_thresholds[pr_threshold_index]
    precision_custom = precision[pr_threshold_index]
    recall_custom = recall[pr_threshold_index]

    def display_confusion_matrix(y_true, y_pred, labels):
        cm = confusion_matrix(y_true, y_pred)
        disp = ConfusionMatrixDisplay(confusion_matrix=cm, display_labels=labels)
        disp.plot(cmap='Blues', ax=ax3, xticks_rotation='vertical', values_format='d')
        ax3.set_xticklabels(ax3.get_xticklabels(), fontsize=8)
        ax3.set_yticklabels(ax3.get_yticklabels(), fontsize=8)
        plt.xlabel('Predicted', fontsize=8)
        plt.ylabel('True', fontsize=8)
        ax3.legend(fontsize=8)

    fig, (ax1, ax2, ax3) = plt.subplots(nrows=1, ncols=3, figsize=(15, 4.5))
    

    ax1.plot(fpr, tpr, color='blue', label='ROC AUC ≈ {:.2f}'.format(roc_auc))
    ax1.plot([0, 1], [0, 1], color='grey', linestyle='--')
    ax1.scatter(fpr_custom, tpr_custom, color='red', label=f'Threshold = {threshold_custom:.2f}', s=50, marker='x')
    # ax1.scatter(fpr_0_5, tpr_0_5, color='red', label='Threshold = {:.2f}'.format(threshold_0_5), s=50, marker='x')
    ax1.set_xlim([0, 1])
    ax1.set_ylim([0, 1])
    ax1.set_xlabel('False Positive Rate', fontsize=8)
    ax1.set_ylabel('True Positive Rate', fontsize=8)
    ax1.set_title('ROC curve', fontsize=8)
    ax1.legend(loc="lower right", fontsize=8)

    chance_level_precision = np.sum(y_test) / len(y_test)
    
    ax2.plot(recall, precision, color='green', label='PR AUC ≈ {:.2f}'.format(pr_auc))
    # ax2.scatter(recall_0_5, precision_0_5, color='orange', label='Threshold = {:.2f}'.format(pr_threshold_0_5), s=50, marker='x')
    ax2.scatter(recall_custom, precision_custom, color='orange', label=f'Threshold = {pr_threshold_custom:.2f}', s=50, marker='x')
    ax2.axhline(y=chance_level_precision, color='grey', linestyle='--', label='Chance Level')
    ax2.set_xlim([0, 1])
    ax2.set_ylim([0, 1])
    ax2.set_xlabel('Recall', fontsize=8)
    ax2.set_ylabel('Precision', fontsize=8)
    ax2.set_title('Precision-Recall curve', fontsize=8)
    ax2.legend(loc="upper right", fontsize=8)

    # ax2.legend(loc="lower left", fontsize=8)

    display_confusion_matrix(y_test, predictions_class, labels=class_labels)
    ax3.set_title('Confusion matrix', fontsize=8)

    plt.subplots_adjust(wspace=0.5)

    plt.rcParams['font.family'] = 'sans-serif'
    plt.rcParams['font.size'] = 8

    plt.savefig(filename, dpi=300)

    print(f'Threshold closest to {threshold} (ROC): {threshold_custom:.2f}')
    print(f'Threshold closest to {threshold} (PR): {pr_threshold_custom:.2f}')

    return results_df

##### function to generate ROC and confusion matrix figures

In [ ]:
# import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
import lightgbm as lgb
import catboost as cb
from sklearn.linear_model import LogisticRegression
import numpy as np
from sklearn.model_selection import StratifiedKFold

# Train each model on the training set.
X_train = devset_imputed.drop(outcome_var, axis=1)
y_train = devset_imputed[outcome_var]
X_test = testset_imputed.drop(outcome_var, axis=1)
y_test = testset_imputed[outcome_var]

devset_withmissing.loc[:, outcome_var] = devset_withmissing[outcome_var].replace({"positive": True, "negative": False}).astype(bool)
testset_withmissing.loc[:, outcome_var] = testset_withmissing[outcome_var].replace({"positive": True, "negative": False}).astype(bool)

# Train each model on the training set.
X_train_withmissing = devset_withmissing.drop(outcome_var, axis=1)
X_test_withmissing = testset_withmissing.drop(outcome_var, axis=1)

# Convert categories to str type
X_train_withmissing_cat = X_train_withmissing.copy()
X_test_withmissing_cat = X_test_withmissing.copy()
X_train_withmissing_cat[cat_features] = X_train_withmissing_cat[cat_features].apply(lambda x: x.astype(str))
X_test_withmissing_cat[cat_features] = X_test_withmissing_cat[cat_features].apply(lambda x: x.astype(str))

# Combine X_train and X_test into a single DataFrame
combined = pd.concat([X_train, X_test], axis=0)
# Loop through the columns of the combined DataFrame and check their data types
for col in combined.columns:
    if combined[col].dtype == 'object' or combined[col].dtype.name == 'category':
        # Perform one-hot encoding on the column
        combined = pd.get_dummies(combined, columns=[col], prefix=[col], drop_first=True)
# when performing one-hot encoding, one of the one-hot encoded columns is redundant and should be omitted. To achieve this, you can use the drop_first=True parameter in the get_dummies() function.

# Split the combined DataFrame back into X_train and X_test
X_train_OHE = combined[:len(X_train)]
X_test_OHE = combined[len(X_train):]


##### variable type encoding for QLattice model

In [ ]:
# import pandas as pd

# create an empty dictionary to store the stypes
stypes = {}

# iterate over each column in the dataset
for col in devset_imputed.columns:
    # check if the column dtype is 'category'
    if pd.api.types.is_categorical_dtype(devset_imputed[col]):
        # if it is, add the column name to the stypes dictionary with a value of 'c'
        stypes[col] = 'c'

stypes[outcome_var] = 'b'
# print the stypes dictionary
print(stypes)



#### set model weights based on class balance from the development set

In [ ]:
from sklearn.utils.class_weight import compute_sample_weight

sample_weights = compute_sample_weight(class_weight='balanced', y=devset_imputed[outcome_var])

## Initiate machine learning models

### QLattice model

In [ ]:
import feyn
ql = feyn.QLattice(random_seed=SEED)

##### cross validation

In [ ]:
import pandas as pd
import numpy as np
from sklearn.model_selection import StratifiedKFold
from sklearn.metrics import confusion_matrix, roc_auc_score, brier_score_loss, precision_recall_curve, auc, matthews_corrcoef
import feyn
from scipy.stats import uniform, randint


def solve(m1, m2, std1, std2):
    a = 1/(2*std1**2) - 1/(2*std2**2)
    b = m2/(std2**2) - m1/(std1**2)
    c = m1**2 /(2*std1**2) - m2**2 / (2*std2**2) - np.log(std2/std1)
    return np.roots([a, b, c])

def find_closest_to_default(roots, default=0.5):
    roots = np.asarray(roots)
    closest_index = np.argmin(np.abs(roots - default))
    return roots[closest_index]

def calculate_metrics(y_true, y_pred, y_pred_proba):
    cm = confusion_matrix(y_true, y_pred)
    tn, fp, fn, tp = cm.ravel()
    PPV = tp / (tp + fp) 
    NPV = tn / (tn + fn) 
    Sensitivity = tp / (tp + fn) 
    Specificity = tn / (tn + fp) 
    Balanced_Accuracy = (Sensitivity + Specificity) / 2
    MCC = matthews_corrcoef(y_true, y_pred)
    ROC_AUC = roc_auc_score(y_true, y_pred_proba)
    precision, recall, _ = precision_recall_curve(y_true, y_pred_proba)
    PR_AUC = auc(recall, precision)
    Brier_Score = brier_score_loss(y_true, y_pred_proba)

    return {
        'PPV': PPV,
        'NPV': NPV,
        'Sensitivity': Sensitivity,
        'Specificity': Specificity,
        'Balanced Accuracy': Balanced_Accuracy,
        'MCC': MCC,
        'ROCAUC': ROC_AUC,
        'PRAUC': PR_AUC,
        'Brier Score': Brier_Score
    }

def cross_validate_model(model_class, X, y, sample_weights=None, n_splits=5, random_state=None, measures=None, use_default_threshold=False, **model_params):
    if measures is None:
        measures = ['PPV', 'NPV', 'Sensitivity', 'Specificity', 'Balanced Accuracy', 'MCC', 'ROCAUC', 'PRAUC', 'Brier Score']

    fold_results = pd.DataFrame()
    aggregated_predictions = np.array([])
    aggregated_labels = np.array([])
    skf = StratifiedKFold(n_splits=n_splits, random_state=random_state, shuffle=True)

    for fold, (train_index, test_index) in enumerate(skf.split(X, y), 1):
        X_train_fold, X_test_fold = X.iloc[train_index], X.iloc[test_index]
        y_train_fold, y_test_fold = y.iloc[train_index], y.iloc[test_index]
        sample_weights_fold = sample_weights[train_index] if sample_weights is not None else None

        # Check if the model is a BalancedRandomForestClassifier
        if model_class == BalancedRandomForestClassifier:
            # Define the corrected parameter grid for random search
            param_dist = {
                'n_estimators': [50, 100, 150, 200],
                'max_depth': [None, 10, 20, 30],
                'min_samples_split': [2, 5, 10],
                'min_samples_leaf': [1, 2, 4],
                'max_features': ['sqrt', 'log2', None],  # Corrected 'auto' to 'sqrt'
            }
            
            # Explicitly set sampling_strategy to 'all'
            model = BalancedRandomForestClassifier(random_state=SEED, sampling_strategy='all',n_jobs=n_cpu_model_training)
            
            random_search = RandomizedSearchCV(
                estimator=model, 
                param_distributions=param_dist, 
                n_iter=n_iter,
                scoring=tun_score, 
                cv=5,
                refit=True,
                random_state=random_state,
                verbose=0, n_jobs = n_cpu_for_tuning)
            # Fit the RandomizedSearchCV object to the data
            random_search.fit(X_train_fold, y_train_fold, sample_weight=sample_weights_fold)
            
            # Get the best parameters and best estimator from the random search
            best_params = random_search.best_params_
            model = BalancedRandomForestClassifier(random_state=SEED, sampling_strategy='all',n_jobs=n_cpu_model_training, **best_params)
            # model = random_search.best_estimator_

            # Fit the best estimator on the entire training data
            model.fit(X_train_fold, y_train_fold, sample_weight=sample_weights_fold)

            # Get predictions on the test data
            predictions_proba = model.predict_proba(X_test_fold)[:, 1]

                # Check if the model is a QLattice
        elif model_class == 'QLattice':
            X_train_fold_ql = X_train_fold.copy()
            X_train_fold_ql[outcome_var] = y_train_fold
            from sklearn.metrics import balanced_accuracy_score
            from joblib import Parallel, delayed
        
            best_balanced_accuracy = 0
            best_parameters = {'n_epochs': 50, 'max_complexity': 10}
        
            def evaluate_params(n_epochs, max_complexity):
                ql = feyn.QLattice(random_seed=random_state)
                models = ql.auto_run(
                    data=X_train_fold_ql,
                    output_name=outcome_var,
                    kind='classification',
                    n_epochs=n_epochs,
                    stypes=stypes,
                    criterion="aic",
                    loss_function='binary_cross_entropy',
                    max_complexity=max_complexity,
                    sample_weights=sample_weights_fold
                )
                best_model = models[0]
                
                predictions_proba = best_model.predict(X_test_fold)
                balanced_acc = balanced_accuracy_score(y_test_fold, (predictions_proba > 0.5).astype(int))
                return balanced_acc, {'n_epochs': n_epochs, 'max_complexity': max_complexity}
        
            results = Parallel(n_jobs=n_cpu_for_tuning)(
                delayed(evaluate_params)(n_epochs, max_complexity)
                for n_epochs in [50, 100]
                for max_complexity in [5, 10]
            )
        
            for balanced_acc, params in results:
                if balanced_acc > best_balanced_accuracy:
                    best_balanced_accuracy = balanced_acc
                    best_parameters = params

            
            print("Best Parameters:", best_parameters)
            print("Best Balanced Accuracy:", best_balanced_accuracy)
                # Use the best parameters from the grid search
            best_n_epochs = best_parameters['n_epochs']
            best_max_complexity = best_parameters['max_complexity']
            
            # Train the final model with the best parameters
            ql = feyn.QLattice(random_seed=random_state)
            models = ql.auto_run(
                data=X_train_fold_ql,
                output_name=outcome_var,
                kind='classification',
                n_epochs=best_n_epochs,
                stypes=stypes,
                criterion="aic",
                loss_function='binary_cross_entropy',
                max_complexity=best_max_complexity,
                sample_weights=sample_weights_fold
            )

            model = models[0]
            predictions_proba = model.predict(X_test_fold)

        elif model_class == HistGradientBoostingClassifier:
            # Create a HistGradientBoostingClassifier instance
            model = HistGradientBoostingClassifier(random_state=random_state, early_stopping=True, validation_fraction=0.2)
            param_dist = {
                'learning_rate': [0.01, 0.05, 0.1 ,0.2],
                'max_depth': [None, 3, 15, 100],
                'min_samples_leaf': [10, 20, 40],
                'max_leaf_nodes': [10, 20, 50],
                'l2_regularization': [0.01, 0.05, 0.1, 0.2],
                "max_iter": [500, 1000]
            }
            # Create a RandomizedSearchCV instance
            random_search = RandomizedSearchCV(
                estimator=model, 
                param_distributions=param_dist, 
                n_iter=n_iter,
                scoring=tun_score, #'roc_auc',
                cv=5,
                refit=True,
                random_state=random_state,
                verbose=0,
            n_jobs = n_cpu_for_tuning)

            # Perform the random search on the training data
            random_search.fit(X_train_fold, y_train_fold, sample_weight=sample_weights_fold)

            # Get the best parameters and best estimator
            best_params = random_search.best_params_
            model = HistGradientBoostingClassifier(random_state=random_state, early_stopping=True, validation_fraction=0.2, **best_params)
            # model = random_search.best_estimator_

            # Fit the best estimator on the entire training data
            model.fit(X_train_fold, y_train_fold, sample_weight=sample_weights_fold)

            # Get predictions on the test data
            predictions_proba = model.predict_proba(X_test_fold)[:, 1]


        elif model_class == lgb.LGBMClassifier:
            param_dist = {
                'num_leaves': randint(6, 50),
                'min_child_samples': randint(100, 500),
                'min_child_weight': [1e-5, 1e-3, 1e-2, 1e-1, 1, 1e1, 1e2, 1e3, 1e4],
                'subsample': uniform(loc=0.2, scale=0.8),
                'colsample_bytree': uniform(loc=0.4, scale=0.6),
                'reg_alpha': [0, 1e-1, 1, 2, 5, 7, 10, 50, 100],
                'reg_lambda': [0, 1e-1, 1, 5, 10, 20, 50, 100],
                'max_depth': randint(3, 15)  
            }

            model = lgb.LGBMClassifier(random_state=random_state,
                                       n_estimators=500,
                                       learning_rate=0.1,
                                       verbosity=verbosity_option,
                                       n_jobs=n_cpu_model_training) 
            random_search = RandomizedSearchCV(
                estimator=model, 
                param_distributions=param_dist, 
                n_iter=n_iter,
                scoring=tun_score, #'roc_auc',
                cv=5,
                refit=True,
                random_state=random_state,
                verbose=0,n_jobs = n_cpu_for_tuning)
           
            # Perform the random search on the training data
            random_search.fit(X_train_fold, y_train_fold, sample_weight=sample_weights_fold)

            # Get the best parameters and best estimator
            best_params = random_search.best_params_
            model = lgb.LGBMClassifier(random_state=random_state,
                                       n_estimators=500,
                                       learning_rate=0.1,
                                       verbosity=verbosity_option,
                                       n_jobs=n_cpu_model_training,
                                       **best_params)

            # Fit the best estimator on the entire training data
            model.fit(X_train_fold, y_train_fold, sample_weight=sample_weights_fold)

            # Get predictions on the test data
            predictions_proba = model.predict_proba(X_test_fold)[:, 1]

        elif model_class == cb.CatBoostClassifier:
            
            # Define the CatBoost classifier
            # Check if there are categorical features
            if len(cat_features) > 0:
                # If there are categorical features, use them
                model = cb.CatBoostClassifier(random_state=SEED, cat_features=cat_features, verbose=0) # verbosity for cb can't be negative
            else:
                # If there are no categorical features, do not use cat_features parameter
                model = cb.CatBoostClassifier(random_state=SEED, verbose=0)
            
            # Define the parameter grid for random search
            param_dist = {
                'learning_rate': np.logspace(-3, 0, num=10),
                'depth': np.arange(4, 11),
                'l2_leaf_reg': np.logspace(-1, 3, num=10),
                'iterations': np.arange(100, 1000, 10),
                'subsample': np.linspace(0.5, 1, 10),
                'random_strength': np.linspace(0, 10, 10)
            }

            # Create a RandomizedSearchCV instance
            random_search = RandomizedSearchCV(
                estimator=model, 
                param_distributions=param_dist, 
                n_iter=n_iter,
                scoring=tun_score, # balanced accuracy had issues for catboost
                cv=5,
                refit=True,
                random_state=random_state,
                verbose=0,n_jobs = n_cpu_for_tuning
            )

            # Perform the random search on the training data
            random_search.fit(X_train_fold, y_train_fold, sample_weight=sample_weights_fold)

            # Get the best parameters and best estimator
            best_params = random_search.best_params_
            if len(cat_features) > 0:
                # If there are categorical features, use them
                model = cb.CatBoostClassifier(random_state=SEED, cat_features=cat_features, verbose=0, **best_params)
            else:
                # If there are no categorical features, do not use cat_features parameter
                model = cb.CatBoostClassifier(random_state=SEED, verbose=0, **best_params)

            # Fit the best estimator on the entire training data
            model.fit(X_train_fold, y_train_fold, sample_weight=sample_weights_fold)

            # Get predictions on the test data
            predictions_proba = model.predict_proba(X_test_fold)[:, 1]

        # Check if the specified model class is LogisticRegression
        elif model_class == LogisticRegression:
                    
            # Define the Logistic Regression classifier
            model = LogisticRegression(random_state=random_state)
            param_dist = {
            'penalty': ['l2', 'none'],
            'C': [0.01, 0.1, 1],
            'max_iter': [500, 1000, 2000],
            'solver': ['newton-cg', 'lbfgs', 'liblinear', 'sag', 'saga'],
            'class_weight': ['balanced', None]}
        
            # Create a RandomizedSearchCV instance
            random_search = RandomizedSearchCV(
                estimator=model, 
                param_distributions=param_dist, 
                n_iter=n_iter,
                scoring=tun_score, #'roc_auc',
                cv=5,
                refit=True,
                random_state=random_state,
                verbose=0, n_jobs=n_cpu_for_tuning
            )
        
            # Perform the random search on the training data
            random_search.fit(X_train_fold, y_train_fold, sample_weight=sample_weights_fold)
        
            # Get the best parameters and best estimator
            best_params = random_search.best_params_
            model = LogisticRegression(random_state=random_state, **best_params)
        
            # Fit the best estimator on the entire training data
            model.fit(X_train_fold, y_train_fold, sample_weight=sample_weights_fold)
        
            # Get predictions on the test data
            predictions_proba = model.predict_proba(X_test_fold)[:, 1]
        
        else:
            model = model_class(**model_params)
            model.fit(X_train_fold, y_train_fold, sample_weight=sample_weights_fold)
            predictions_proba = model.predict_proba(X_test_fold)[:, 1]

        # Aggregate predictions and labels
        aggregated_predictions = np.concatenate((aggregated_predictions, predictions_proba))
        aggregated_labels = np.concatenate((aggregated_labels, y_test_fold))

    # Estimate parameters and find optimal threshold
    m1, std1 = np.mean(aggregated_predictions[aggregated_labels == False]), np.std(aggregated_predictions[aggregated_labels == False])
    m2, std2 = np.mean(aggregated_predictions[aggregated_labels == True]), np.std(aggregated_predictions[aggregated_labels == True])
    intersections = solve(m1, m2, std1, std2)

        # Use default threshold if specified
    if use_default_threshold:
        optimal_threshold = 0.5
    else:
        optimal_threshold = find_closest_to_default(intersections)

    # Calculate and store metrics for each fold
    for fold, (train_index, test_index) in enumerate(skf.split(X, y), 1):
        X_test_fold = X.iloc[test_index]
        y_test_fold = y.iloc[test_index]

        # Get predictions for the current fold using the optimal threshold
        if model_class == 'QLattice':
            predictions_proba_fold = model.predict(X_test_fold)
        else:
            predictions_proba_fold = model.predict_proba(X_test_fold)[:, 1]

        predictions_class_fold = np.where(predictions_proba_fold >= optimal_threshold, 1, 0)
        
        # Calculate metrics
        metrics = calculate_metrics(y_test_fold, predictions_class_fold, predictions_proba_fold)
        metrics['Fold'] = fold
        # fold_results = fold_results.append(metrics, ignore_index=True)
        fold_results = pd.concat([fold_results, pd.DataFrame(metrics, index=[0])], ignore_index=True)


    # Aggregate results across folds
    aggregated_results = {metric: np.nanmean(fold_results[metric].values).round(2) for metric in measures}
    aggregated_results_sd = {metric: np.nanstd(fold_results[metric].values).round(2) for metric in measures}

    # Combining mean and standard deviation
    combined_results = {metric: f"{mean} ± {sd}" for metric, mean in aggregated_results.items() for _, sd in aggregated_results_sd.items() if metric == _}
    
    # Creating a DataFrame for tabular display
    results_table = pd.DataFrame(list(combined_results.items()), columns=['Metric', 'Result'])

    # Displaying the results
    print("Aggregated Results:")
    print(results_table.to_string(index=False))

    return fold_results, results_table, optimal_threshold



In [ ]:
import feyn
fold_results_QLattice, aggregated_results_formatted_QLattice, optimal_threshold_QLattice = cross_validate_model(model_class='QLattice',
                                                                                                                X=X_train, 
                                                                                                                y=y_train, 
                                                                                                                sample_weights=sample_weights, 
                                                                                                                random_state=SEED,
                                                                                                               use_default_threshold = use_default_threshold)

#### QLattice model development using the whole development set

In [ ]:
from sklearn.metrics import balanced_accuracy_score

best_balanced_accuracy = 0
best_parameters = {'n_epochs': 50, 'max_complexity': 10}

# Specify the grid search space
epoch_values = [50, 100]
complexity_values = [5, 10]

for n_epochs in epoch_values:
    for max_complexity in complexity_values:
        # Update the parameters for the QLattice model
        ql_params = {
            'n_epochs': n_epochs,
            'max_complexity': max_complexity,
            'stypes': stypes,
            'criterion': "aic",
            'loss_function': 'binary_cross_entropy',
            'sample_weights': sample_weights
        }

        # Create the QLattice model
        ql = feyn.QLattice(random_seed=SEED)
        models = ql.auto_run(data=devset_imputed, output_name=outcome_var, kind='classification', **ql_params)
        best_model = models[0]

        # Make predictions on the test set
        predictions_proba = best_model.predict(devset_imputed)

        # Calculate balanced accuracy
        balanced_acc = balanced_accuracy_score(y_train, (predictions_proba > 0.5).astype(int))

        # Check if the current parameters yield a better balanced accuracy
        if balanced_acc > best_balanced_accuracy:
            best_balanced_accuracy = balanced_acc
            best_parameters = {'n_epochs': n_epochs, 'max_complexity': max_complexity}

print("Best Parameters:", best_parameters)
print("Best Balanced Accuracy:", best_balanced_accuracy)

# Use the best parameters to train the final model
best_n_epochs = best_parameters['n_epochs']
best_max_complexity = best_parameters['max_complexity']

final_ql = feyn.QLattice(random_seed=SEED)
models = final_ql.auto_run(
    data=devset_imputed,
    output_name=outcome_var,
    kind='classification',
    n_epochs=best_n_epochs,
    stypes=stypes,
    criterion="aic",
    loss_function='binary_cross_entropy',
    max_complexity=best_max_complexity,
    sample_weights=sample_weights
)



In [ ]:
best_model = models[0]

#### associations of the QLattice model variables

In [ ]:
best_model.plot_signal(devset_imputed,corr_func='spearman')

In [ ]:
best_model.plot_signal(testset_imputed,corr_func='spearman')

In [ ]:
best_model.plot_signal(devset_imputed,corr_func='mutual_information')

In [ ]:
best_model.plot_signal(testset_imputed,corr_func='mutual_information')

#### QLattice model performance on the test set

In [ ]:
results_df_QLattice = evaluate_and_plot_model(model = best_model,
                                              threshold = optimal_threshold_QLattice,
                                              testset = testset_imputed,
                                              y_test = y_test,
                                              class_labels = ['Negative', 'Positive'], filename='ROC_CM_QLattice.png')

#### QLattice model overview

In [ ]:
best_model.plot(devset_imputed, testset_imputed)

In [ ]:
# feature selected by the model
best_model.features

#### distribution of model predicted probabilities for each class

In [ ]:
# import matplotlib.pyplot as plt
best_model.plot_probability_scores(testset_imputed)


#### model representation as a closed-form expression

In [ ]:
sympy_model = best_model.sympify(symbolic_lr=True, include_weights=True)

sympy_model.as_expr()

In [ ]:
cat_features

### stratified 5-fold cross validation 

Here we do cross validation to see how the model may perform on the test set.
This is done for each of the laternative models that is Random Forest, LightGBM, CATBoost, and Logistic Regression

In [ ]:
from sklearn.naive_bayes import GaussianNB
fold_results_NB, aggregated_results_formatted_NB, optimal_threshold_NB = cross_validate_model(model_class=GaussianNB,
                                                                                              X=X_train_OHE,
                                                                                              y=y_train,
                                                                                              sample_weights=sample_weights,
                                                                                              random_state=SEED,
                                                                                             use_default_threshold = use_default_threshold)

In [ ]:
from sklearn.linear_model import LogisticRegression
fold_results_LR, aggregated_results_formatted_LR, optimal_threshold_LR = cross_validate_model(model_class=LogisticRegression,
                                                                                              X=X_train_OHE, 
                                                                                              y=y_train,
                                                                                              sample_weights=sample_weights,
                                                                                              random_state=SEED, 
                                                                                             use_default_threshold = use_default_threshold)

##### Balanced Random Forest Classifier (RBF)

In [ ]:
from imblearn.ensemble import BalancedRandomForestClassifier
fold_results_BRF, aggregated_results_formatted_BRF, optimal_threshold_BRF = cross_validate_model(model_class=BalancedRandomForestClassifier,
                                                                                                 X = X_train_OHE,
                                                                                                 y = y_train,
                                                                                                 sample_weights = sample_weights,
                                                                                                 random_state = SEED,
                                                                                                use_default_threshold = use_default_threshold)

##### Histogram-based Gradient Boosting Classification Tree (HGBC)

In [ ]:
from sklearn.ensemble import HistGradientBoostingClassifier
fold_results_HGBC, aggregated_results_formatted_HGBC, optimal_threshold_HGBC = cross_validate_model(model_class=HistGradientBoostingClassifier,
                                                                                                    X = X_train_OHE,
                                                                                                    y = y_train,
                                                                                                    sample_weights = sample_weights,
                                                                                                    random_state = SEED,
                                                                                                   use_default_threshold = use_default_threshold)

##### light gradient-boosting machine (LightGBM)

In [ ]:
import lightgbm as lgb
fold_results_LGBM, aggregated_results_formatted_LGBM, optimal_threshold_LGBM = cross_validate_model(model_class=lgb.LGBMClassifier,
                                                                                                    X = X_train_withmissing,
                                                                                                    y = y_train,
                                                                                                    sample_weights = sample_weights,
                                                                                                    random_state = SEED,
                                                                                                   use_default_threshold = use_default_threshold)

##### categorical boosting (CATBoost)

In [ ]:
cat_features = list(cat_features)

In [ ]:
import catboost as cb
fold_results_CB, aggregated_results_formatted_CB, optimal_threshold_CB = cross_validate_model(model_class=cb.CatBoostClassifier,
                                                                                              X = X_train_withmissing,
                                                                                              y = y_train,
                                                                                              sample_weights = sample_weights,
                                                                                              random_state = SEED,
                                                                                              cat_features=cat_features, 
                                                                                              # iterations=500, 
                                                                                              verbose=0,
                                                                                             use_default_threshold = use_default_threshold)

In [ ]:
models = [aggregated_results_formatted_QLattice, aggregated_results_formatted_LR, aggregated_results_formatted_BRF,
          aggregated_results_formatted_HGBC, aggregated_results_formatted_LGBM, aggregated_results_formatted_CB,
          aggregated_results_formatted_NB]

# Set 'Metric' as the index for each model's DataFrame
for model in models:
    model.set_index('Metric', inplace=True)

# Concatenate the DataFrames along the columns
aggregated_results_formatted_all = pd.concat(models, axis=1)

# Set the column names
aggregated_results_formatted_all.columns = ["QLattice", "LR", "BRF", "HGBC", "LGBM", "CB", "NB"]

# Display the results
print(aggregated_results_formatted_all)

# Save the results to an Excel file
aggregated_results_formatted_all.to_excel('aggregated_results_formatted_all.xlsx', index=True)


### Model training using the whole training set

In [ ]:
from sklearn.naive_bayes import GaussianNB
from imblearn.ensemble import BalancedRandomForestClassifier
from sklearn.ensemble import HistGradientBoostingClassifier
import lightgbm as lgb
import catboost as cb
from sklearn.linear_model import LogisticRegression


In [ ]:
from sklearn.dummy import DummyClassifier

# Train Dummy Classifier
dummy_classifier = DummyClassifier(strategy='most_frequent')  
dummy_classifier.fit(X_train_OHE, y_train, sample_weight=sample_weights)

results_df_dummy = evaluate_and_plot_model(model = dummy_classifier,
                                        threshold = 0.5,
                                        testset = X_test_OHE,
                                        y_test = y_test,
                                        class_labels = ['Negative', 'Positive'],
                                        filename='ROC_CM_dummy_most_frequent.png')

In [ ]:
from sklearn.dummy import DummyClassifier

# Train Dummy Classifier with 'stratified' strategy
dummy_classifier = DummyClassifier(strategy='stratified')
dummy_classifier.fit(X_train_OHE, y_train, sample_weight=sample_weights)

results_df_dummy = evaluate_and_plot_model(model = dummy_classifier,
                                        threshold = 0.5,
                                        testset = X_test_OHE,
                                        y_test = y_test,
                                        class_labels = ['Negative', 'Positive'],
                                        filename='ROC_CM_dummy_stratified.png')

### evaluate alternative models on the test set

##### Naive Bayes

In [ ]:
# Train Naive Bayes
nb_classifier = GaussianNB()
nb_classifier.fit(X_train_OHE, y_train, sample_weight=sample_weights)

results_df_NB = evaluate_and_plot_model(model = nb_classifier, threshold = optimal_threshold_NB, testset = X_test_OHE, y_test = y_test, class_labels = ['Negative', 'Positive'], filename='ROC_CM_NB.png')

##### Logistic Regression

In [ ]:
from sklearn.linear_model import LogisticRegression

# Define the parameter grid for random search
param_dist = {
            'penalty': ['l2', 'none'],
            'C': [0.01, 0.1, 1],
            'max_iter': [500, 1000, 2000],
            'solver': ['newton-cg', 'lbfgs', 'liblinear', 'sag', 'saga'],
            'class_weight': ['balanced', None]}

# Create a Logistic Regression instance
lr = LogisticRegression(random_state=SEED)

# Create a RandomizedSearchCV instance
random_search = RandomizedSearchCV(
    estimator=lr,
    param_distributions=param_dist,
    n_iter=n_iter, 
    scoring=tun_score,
    cv=5,
    refit=True,
    random_state=SEED,
    verbose=0,
    n_jobs=n_cpu_for_tuning
)

# Perform the random search on the training data
random_search.fit(X_train_OHE, y_train, sample_weight=sample_weights)

# Get the best parameters and best estimator
best_params = random_search.best_params_
lr = LogisticRegression(random_state=SEED, **best_params)

# Fit the best estimator on the entire training data
lr.fit(X_train_OHE, y_train, sample_weight=sample_weights)


In [ ]:
results_df_LR = evaluate_and_plot_model(model = lr,
                                        threshold = optimal_threshold_LR,
                                        testset = X_test_OHE,
                                        y_test = y_test,
                                        class_labels = ['Negative', 'Positive'],
                                        filename='ROC_CM_LR.png')

##### HGBC

In [ ]:
from sklearn.model_selection import RandomizedSearchCV
from sklearn.experimental import enable_hist_gradient_boosting  
from sklearn.ensemble import HistGradientBoostingClassifier
from sklearn.metrics import make_scorer, balanced_accuracy_score
from scipy.stats import uniform, randint

# Define the parameter grid for random search
param_dist = {
    'learning_rate': [0.01, 0.05, 0.1 ,0.2],
    'max_depth': [None, 3, 15, 100],
    'min_samples_leaf': [10, 20, 40],
    'max_leaf_nodes': [10, 20, 50],
    'l2_regularization': [0.01, 0.05, 0.1, 0.2],
    "max_iter": [500, 1000]
}

# Create a HistGradientBoostingClassifier instance
HGBC = HistGradientBoostingClassifier(random_state=SEED, early_stopping=True, validation_fraction=0.2)

# Create a RandomizedSearchCV instance
random_search = RandomizedSearchCV(
    estimator=HGBC, 
    param_distributions=param_dist, 
    n_iter=n_iter,
    scoring=tun_score,
    cv=5,
    refit=True,
    random_state=SEED,
    verbose=0,n_jobs = n_cpu_for_tuning)

# Perform the random search on the training data
random_search.fit(X_train_OHE, y_train, sample_weight=sample_weights)

# Get the best parameters and best estimator
best_params = random_search.best_params_
HGBC = HistGradientBoostingClassifier(random_state=SEED, early_stopping=True, validation_fraction=0.2, **best_params)

# Fit the best estimator on the entire training data
HGBC.fit(X_train_OHE, y_train, sample_weight=sample_weights)



In [ ]:
results_df_HGBC = evaluate_and_plot_model(model = HGBC, threshold = optimal_threshold_HGBC, testset = X_test_OHE, y_test = y_test, class_labels = ['Negative', 'Positive'], filename='ROC_CM_HGBC.png') 

##### BRF

In [ ]:
from sklearn.model_selection import RandomizedSearchCV
from imblearn.ensemble import BalancedRandomForestClassifier

# Define the parameter grid for random search
param_dist = {
    'n_estimators': [50, 100, 150, 200],
    'max_depth': [None, 10, 20, 30],
    'min_samples_split': [2, 5, 10],
    'min_samples_leaf': [1, 2, 4],
    'max_features': ['sqrt', 'log2', None],
}

# Explicitly set sampling_strategy to 'all'
brf = BalancedRandomForestClassifier(random_state=SEED, sampling_strategy='all', n_jobs=n_cpu_model_training)

# Create RandomizedSearchCV object with balanced accuracy as the scoring metric
random_search = RandomizedSearchCV(
    estimator=brf, 
    param_distributions=param_dist, 
    n_iter=n_iter,
    scoring=tun_score,
    cv=5,
    refit=True,
    random_state=SEED,
    verbose=0,n_jobs = n_cpu_for_tuning)

# Fit the RandomizedSearchCV object to the data
random_search.fit(X_train_OHE, y_train, sample_weight=sample_weights)

# Get the best parameters and best estimator from the random search
best_params = random_search.best_params_

# Reinitialize a new brf model with the best parameters
brf = BalancedRandomForestClassifier(random_state=SEED, sampling_strategy='all',n_jobs=n_cpu_model_training,  **best_params)

# Train the new model on the training data
brf.fit(X_train_OHE, y_train, sample_weight=sample_weights)

# Print the best parameters
print("Best Parameters:", best_params)


In [ ]:
results_df_BRF = evaluate_and_plot_model(model = brf, threshold = optimal_threshold_BRF, testset = X_test_OHE, y_test = y_test, class_labels = ['Negative', 'Positive'], filename='ROC_CM_BRF.png')

##### LGBM

In [ ]:
import lightgbm as lgb
from sklearn.model_selection import RandomizedSearchCV
from sklearn.metrics import make_scorer, balanced_accuracy_score
from scipy.stats import randint as sp_randint

# Define the classifier
lgbm = lgb.LGBMClassifier(random_state=SEED,
                          verbosity=verbosity_option,
                          n_jobs=n_cpu_model_training,
                          n_estimators=500,
                          learning_rate=0.1)


param_dist = {
                'num_leaves': randint(6, 50),
                'min_child_samples': randint(100, 500),
                'min_child_weight': [1e-5, 1e-3, 1e-2, 1e-1, 1, 1e1, 1e2, 1e3, 1e4],
                'subsample': uniform(loc=0.2, scale=0.8),
                'colsample_bytree': uniform(loc=0.4, scale=0.6),
                'reg_alpha': [0, 1e-1, 1, 2, 5, 7, 10, 50, 100],
                'reg_lambda': [0, 1e-1, 1, 5, 10, 20, 50, 100],
                'max_depth': randint(3, 15) 
            }
# Define the search strategy and scoring metric
# Create a RandomizedSearchCV instance
random_search = RandomizedSearchCV(
    estimator=lgbm, 
    param_distributions=param_dist, 
    n_iter=n_iter,
    scoring=tun_score,
    cv=5,
    refit=True,
    random_state=SEED,
    verbose=0,
    n_jobs = n_cpu_for_tuning)

# Perform the random search on the data
random_search.fit(X_train_withmissing, y_train, sample_weight=sample_weights)

# Get the best parameters and best model
best_params = random_search.best_params_

# Print the best parameters
print("Best Hyperparameters:", best_params)

# Reinitialize a new lgbm model with the best parameters
lgbm = lgb.LGBMClassifier(random_state=SEED,
                          **best_params,
                          verbosity=verbosity_option,
                          n_jobs=n_cpu_model_training,
                          n_estimators=500,
                          learning_rate=0.1)

# Train the new model on the training data
lgbm.fit(X_train_withmissing, y_train, sample_weight=sample_weights)


In [ ]:
results_df_LGBM = evaluate_and_plot_model(model = lgbm,
                                          threshold =optimal_threshold_LGBM,
                                          testset = X_test_withmissing,
                                          y_test = y_test,
                                          class_labels = ['Negative', 'Positive'],
                                          filename='ROC_CM_LGBM.png')

##### CatBoost

In [ ]:
import numpy as np
from sklearn.model_selection import RandomizedSearchCV
from sklearn.metrics import make_scorer, balanced_accuracy_score
from catboost import CatBoostClassifier

# Check if there are categorical features
if len(cat_features) > 0:
    # If there are categorical features, use them
    catb = CatBoostClassifier(random_state=SEED, cat_features=cat_features, verbose=0)
else:
    # If there are no categorical features, do not use cat_features parameter
    catb = CatBoostClassifier(random_state=SEED, verbose=0)

# Define the parameter grid for random search
param_dist = {
    'learning_rate': np.logspace(-3, 0, num=10),
    'depth': np.arange(4, 11),
    'l2_leaf_reg': np.logspace(-1, 3, num=10),
    'iterations': np.arange(100, 1000, 10),
    'subsample': np.linspace(0.5, 1, 10),
    'random_strength': np.linspace(0, 10, 10)
}

# Perform random search
random_search = RandomizedSearchCV(
    estimator=catb, 
    param_distributions=param_dist, 
    n_iter=n_iter,
    scoring=tun_score,#scorer,
    cv=5,
    refit=True,
    random_state=SEED,
    verbose=0,
    n_jobs=n_cpu_for_tuning
)

# Fit the random search on your data
random_search.fit(X_train_withmissing, y_train, sample_weight=sample_weights)

# Get the best parameters and best model
best_params = random_search.best_params_

# Check if there are categorical features
if len(cat_features) > 0:
    # If there are categorical features, use them
    catb = CatBoostClassifier(random_state=SEED, cat_features=cat_features, verbose=0, **best_params)
else:
    # If there are no categorical features, do not use cat_features parameter
    catb = CatBoostClassifier(random_state=SEED, verbose=0, **best_params)
    

# Train the new model on the training data
catb.fit(X_train_withmissing, y_train, sample_weight=sample_weights)


In [ ]:
results_df_CB = evaluate_and_plot_model(model = catb,threshold = optimal_threshold_CB, testset = X_test_withmissing, y_test = y_test, class_labels = ['Negative', 'Positive'], filename='ROC_CM_CB.png')

#### Choose the best performing model

In [ ]:
import pandas as pd

# Define a list of models and their respective dataframes
models_and_dfs = [
    ('QLattice', results_df_QLattice),
    ('LR', results_df_LR),
    ('BRF', results_df_BRF),
    ('HGBC', results_df_HGBC),
    ('LGBM', results_df_LGBM),
    ('CB', results_df_CB),
    ("NB", results_df_NB)
]

# Merge data frames based on the "Metric" column
merged_df = models_and_dfs[0][1].copy()

for model, df in models_and_dfs[1:]:
    merged_df = pd.merge(merged_df, df, on='Metric', suffixes=(f'_{model}', ''))

# Rename columns
columns = ['Measures'] + [f'{model}_{col}' for model, _ in models_and_dfs for col in df.columns[1:]]
merged_df.columns = columns

# Copy the merged dataframe
aggregated_results_test_all = merged_df.copy()

# Print the aggregated results
print(aggregated_results_test_all)

df = aggregated_results_test_all.copy()


In [ ]:
# List of metrics
metrics = ['PPV', 'NPV', 'Sensitivity', 'Specificity', 'Balanced Accuracy', 'MCC', 'ROCAUC', 'PRAUC', 'Brier Score']
# List of methods (e.g., QLattice, LR)
methods = ['QLattice', 'LR', 'BRF', 'HGBC', 'LGBM', 'CB', "NB"]  # Add more if you have more methods

# Iterate over each metric and method, merging the columns
for metric in metrics:
    for method in methods:
        columns_to_merge = [f'{method}_Value', f'{method}_Lower Bound', f'{method}_Upper Bound']
        new_column_name = f'{method}'

        # Check if the columns to merge exist in the DataFrame
        if all(col in df.columns for col in columns_to_merge):
            # Merge the columns into a new column
            df[new_column_name] = df[columns_to_merge].apply(lambda x: f'{x[0]} ({x[1]} {x[2]})', axis=1)

            # Optionally, drop the individual columns if needed
            df = df.drop(columns=columns_to_merge)

# Display the resulting DataFrame
print(df)
# Save the results to an Excel file
df.to_excel('aggregated_results_test_withCI_all.xlsx', index=False)

In [ ]:
import pandas as pd

# Define a list of models and their respective dataframes
models_and_dfs = [
    ('QLattice', results_df_QLattice),
    ('LR', results_df_LR),
    ('BRF', results_df_BRF),
    ('HGBC', results_df_HGBC),
    ('LGBM', results_df_LGBM),
    ('CB', results_df_CB),
    ("NB", results_df_NB)
]

# Merge data frames based on the "Metric" column
merged_df = models_and_dfs[0][1][['Metric', 'Value']].copy()

for model, df in models_and_dfs[1:]:
    merged_df = pd.merge(merged_df, df[['Metric', 'Value']], on='Metric', suffixes=(f'_{model}', ''))

# Rename columns
columns = ['Measures'] + [f'{model}' for model, _ in models_and_dfs for col in ['Value']]
merged_df.columns = columns

# Copy the merged dataframe
aggregated_results_test_all = merged_df.copy()

# Print the aggregated results
print(aggregated_results_test_all)

# Save the results to an Excel file
aggregated_results_test_all.to_excel('aggregated_results_test_all.xlsx', index=False)


In [ ]:
data = aggregated_results_test_all.copy()

# replace NA with 0 to calculate the following mean for ranking, correctly
# NA comes from the cases where we have no negative (or positive) class in predictions
data.fillna(0, inplace=True)

# Get the average value for the numerical columns
average_values = data.iloc[5:8, 2:].mean(axis=0) # QLattice excluded from comparison

# Get the name of the one with the highest average
highest_average = average_values.idxmax()

model_dictionary = {"BRF": brf,
                    "HGBC": HGBC,
                    "LGBM": lgbm,
                    "CB": catb,
                    "LR": lr,
                    "NB": nb_classifier
}
print("Selected Model:", highest_average)
selected_model =  model_dictionary[highest_average]

### Model interpretation (for the best alternative model to QLattice)

#### SHAP values association with predicted probabilities

In [ ]:
import matplotlib.pyplot as plt
import shap
from sklearn.naive_bayes import GaussianNB
from sklearn.linear_model import LogisticRegression
import catboost as cb

# Calculate SHAP values for the positive class
positive_class_index = 1  # Adjust this index based on the class labels of your problem

if isinstance(selected_model, (HistGradientBoostingClassifier, BalancedRandomForestClassifier)):
    explainer = shap.TreeExplainer(selected_model,feature_perturbation='interventional')
    shap_values = explainer.shap_values(X_test_OHE)
    if isinstance(selected_model, BalancedRandomForestClassifier):
        shap_values = shap_values[positive_class_index]
elif isinstance(selected_model, cb.CatBoostClassifier):
    explainer = shap.TreeExplainer(selected_model)
    shap_values = explainer.shap_values(X_test_withmissing)
elif isinstance(selected_model, LogisticRegression):
    explainer = shap.LinearExplainer(selected_model, X_train_OHE)
    shap_values = explainer.shap_values(X_test_OHE)
elif isinstance(selected_model, GaussianNB):  
    explainer = shap.KernelExplainer(selected_model.predict_proba, X_train_OHE)
    shap_values = explainer.shap_values(X_test_OHE, nsamples=100)  
    shap_values = shap_values[positive_class_index]
else:
    explainer = shap.TreeExplainer(selected_model)
    shap_values = explainer.shap_values(X_test_withmissing)
    shap_values = shap_values[positive_class_index]

# Calculate the sum of SHAP values for each sample
shap_sum = shap_values.sum(axis=1)

# Get the predicted probabilities of the model
if isinstance(selected_model, (HistGradientBoostingClassifier, BalancedRandomForestClassifier)):
    predicted_probabilities = selected_model.predict_proba(X_test_OHE)[:, positive_class_index]
elif isinstance(selected_model, cb.CatBoostClassifier):
    predicted_probabilities = selected_model.predict_proba(X_test_withmissing)[:, positive_class_index]
elif isinstance(selected_model, (LogisticRegression, GaussianNB)):
    predicted_probabilities = selected_model.predict_proba(X_test_OHE)[:, positive_class_index]
else:
    predicted_probabilities = selected_model.predict_proba(X_test_withmissing)[:, positive_class_index]

# Plot the SHAP sum against the predicted probabilities
plt.scatter(shap_sum, predicted_probabilities)
plt.xlabel('Sum of SHAP values')
plt.ylabel('Predicted Probability')
plt.title('Sum of SHAP Values vs. Predicted Probability')
plt.show()


#### interpret the model based on SHAP analysis

In [ ]:
# Calculate the absolute SHAP values
abs_shap_values = np.abs(shap_values)

# Compute the feature importance based on the sum of absolute SHAP values
feature_importance = np.mean(abs_shap_values, axis=0)

# Create a DataFrame to store feature importance
# feature_importance_df = pd.DataFrame({'Feature': X_train.columns.tolist(), 'Importance': feature_importance})
if isinstance(selected_model, (HistGradientBoostingClassifier, BalancedRandomForestClassifier, LogisticRegression, GaussianNB)):
    feature_importance_df = pd.DataFrame({'Feature': [data_dictionary.get(feature, feature) for feature in X_test_OHE.columns.tolist()], 'Importance': feature_importance})
else:
    feature_importance_df = pd.DataFrame({'Feature': [data_dictionary.get(feature, feature) for feature in X_test_withmissing.columns.tolist()], 'Importance': feature_importance})

# Sort the features by importance in descending order
feature_importance_df = feature_importance_df.sort_values('Importance', ascending=False)

# Print the top 20 most important features 
top_20_features = feature_importance_df.head(20)
print(top_20_features)
# Reverse the order of the sorted data
top_20_features = top_20_features[::-1]

# Reset the index of the DataFrame
top_20_features.reset_index(drop=True, inplace=True)

# Plot the top 20 most important features
plt.figure(figsize=(10, 6))

plt.barh(top_20_features.index, top_20_features['Importance'])
plt.yticks(top_20_features.index, top_20_features['Feature'])
plt.xlabel('Feature Importance')
plt.ylabel('Feature')
plt.title('Top 20 Most Important Features')
plt.rcParams['figure.autolayout'] = True  # Automatically adjust the figure margins
# Save the figure as a PNG file
plt.savefig('feature_importance_shap_plot.png', dpi=300)

plt.show()


#### SHAP summary plot

Note: the plot cannot show categorical features in color codes and thus they are plotted in grey (not mistaken with missing values)

In [ ]:
# Retrieve feature names from the data dictionary
if isinstance(selected_model, (HistGradientBoostingClassifier, BalancedRandomForestClassifier,LogisticRegression, GaussianNB)):
    feature_names_with_shapvalues = [
        data_dictionary.get(feature, feature) + ": " + str(round(value, 2))
        for feature, value in zip(X_test_OHE.columns, np.mean(np.abs(shap_values), axis=0)) 
    ]
    shap.summary_plot(shap_values, X_test_OHE, feature_names=feature_names_with_shapvalues, show=False, alpha = 0.8, max_display=20)
elif isinstance(selected_model, (cb.CatBoostClassifier)):
    feature_names_with_shapvalues = [
        data_dictionary.get(feature, feature) + ": " + str(round(value, 2))
        for feature, value in zip(X_test_withmissing.columns, np.mean(np.abs(shap_values), axis=0)) 
    ]
    shap.summary_plot(shap_values, X_test_withmissing, feature_names=feature_names_with_shapvalues, show=False, alpha = 0.8, max_display=20)
else:
    feature_names_with_shapvalues = [
        data_dictionary.get(feature, feature) + ": " + str(round(value, 2))
        for feature, value in zip(X_test_withmissing.columns, np.mean(np.abs(shap_values), axis=0)) 
    ]
    shap.summary_plot(shap_values, X_test_withmissing, feature_names=feature_names_with_shapvalues, show=False, alpha = 0.8, max_display=20)


# Save the figure as a PNG file
plt.savefig('shap_summary_top20_plot.png', dpi=300)
plt.rcParams['figure.autolayout'] = True  # Automatically adjust the figure margins

# Display the plot
plt.show()



In [ ]:
# Our naive cutoff point is zero log odds (probability 0.5).
# Get predictions for the dev set
if isinstance(selected_model, (HistGradientBoostingClassifier, BalancedRandomForestClassifier, LogisticRegression, GaussianNB)):

    predictions = selected_model.predict_proba(X_test_OHE)
    predictions = predictions[:, 1]
    # print(predictions_class)
    y_pred = [True if x >= 0.5 else False for x in predictions]
    misclassified = y_pred != y_test
elif isinstance(selected_model, (cb.CatBoostClassifier)):
    predictions = selected_model.predict_proba(X_test_withmissing)
    predictions = predictions[:, 1]
    # print(predictions_class)
    y_pred = [True if x >= 0.5 else False for x in predictions]
    misclassified = y_pred != y_test
else:
    predictions = selected_model.predict_proba(X_test_withmissing)
    predictions = predictions[:, 1]
    # print(predictions_class)
    y_pred = [True if x >= 0.5 else False for x in predictions]
    misclassified = y_pred != y_test


#### SHAP decision plot 
only significantly contributed features could also be plotted

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import shap

# Plot the SHAP decision plot with only significant features
if isinstance(selected_model, (HistGradientBoostingClassifier)):
    shap.decision_plot(explainer.expected_value, 
                       shap_values,
                       X_test_OHE,
                       alpha=0.5, feature_names=feature_names_with_shapvalues,
                       link='logit',
                       highlight=misclassified,ignore_warnings=True)
elif isinstance(selected_model, (BalancedRandomForestClassifier)):
    shap.decision_plot(explainer.expected_value[positive_class_index], 
                       shap_values,
                       X_test_OHE,
                       alpha=0.5, feature_names=feature_names_with_shapvalues,
                       link='logit',
                       highlight=misclassified,ignore_warnings=True)
elif isinstance(selected_model, (LogisticRegression)):
    feature_names_with_shapvalues = [
        data_dictionary.get(feature, feature) + ": " + str(round(value, 2))
        for feature, value in zip(X_test_OHE.columns, np.mean(np.abs(shap_values), axis=0)) 
    ]
    shap.decision_plot(explainer.expected_value, 
                       shap_values,
                       X_test_OHE,
                       alpha=0.5, 
                       feature_names=feature_names_with_shapvalues,
                       link='logit',
                       highlight=misclassified,ignore_warnings=True)
elif isinstance(selected_model, (cb.CatBoostClassifier)):
    shap.decision_plot(explainer.expected_value, 
                       shap_values,
                       X_test_withmissing,
                       alpha=0.5, feature_names=feature_names_with_shapvalues,
                       link='logit',
                       highlight=misclassified,ignore_warnings=True)
elif isinstance(selected_model, (GaussianNB)):
    feature_names_with_shapvalues = [
        data_dictionary.get(feature, feature) + ": " + str(round(value, 2))
        for feature, value in zip(X_test_OHE.columns, np.mean(np.abs(shap_values), axis=0))
    ]
    shap.decision_plot(explainer.expected_value[positive_class_index], 
                       shap_values,
                       X_test_OHE,
                       alpha=0.5, 
                       feature_names=feature_names_with_shapvalues,
                       link='logit',
                       highlight=misclassified, ignore_warnings=True)
else:
    shap.decision_plot(explainer.expected_value[positive_class_index], 
                       shap_values,
                       X_test_withmissing,
                       alpha=0.5, feature_names=feature_names_with_shapvalues,
                       link='logit',
                       highlight=misclassified, ignore_warnings=True)

# Save the figure as a PNG file
plt.savefig('shap_decision_allfeats_plot.png', dpi=300)
plt.rcParams['figure.autolayout'] = True  # Automatically adjust the figure margins

# Display the plots
plt.show()


In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import shap
from scipy.stats import spearmanr, ttest_rel
import seaborn as sns
from sklearn.preprocessing import LabelEncoder
from scipy.stats import linregress
# Set the style to emulate the Nature journal style
sns.set(style="ticks")

# Set the background color to white
sns.set_style("white")

plt.rcParams["figure.figsize"] = (10, 5)

# Compute median absolute SHAP values for each feature
median_abs_shap_values = np.median(np.abs(shap_values), axis=0)

# Sort features by median absolute SHAP values in descending order
sorted_features = np.argsort(median_abs_shap_values)[::-1]

# Calculate the number of features to plot
num_features_to_plot = min(np.sum(median_abs_shap_values > 0), 20)  # Plot up to 10 features

# Set the number of columns for subplots
num_cols = 3

# Calculate the number of rows for subplots
num_rows = int(np.ceil(num_features_to_plot / num_cols))

# Initialize a subplot
fig, axs = plt.subplots(num_rows, num_cols, figsize=(15, 5*num_rows))
axs = axs.ravel()

# Track the current subplot index
current_subplot = 0

# Iterate over the top features
for feature in sorted_features[:num_features_to_plot]:
    # Get feature name
    if isinstance(selected_model, (HistGradientBoostingClassifier, BalancedRandomForestClassifier, LogisticRegression, GaussianNB)):
        feature_name = X_test_OHE.columns[feature]
        x = X_test_OHE.iloc[:, feature]
    else:
        feature_name = X_test_withmissing.columns[feature]
        if X_test[feature_name].dtype.name == 'category':
            # Convert categorical feature to numerical using LabelEncoder
            encoder = LabelEncoder()
            X_test_encoded = X_test_withmissing.copy()
            X_test_encoded[feature_name] = encoder.fit_transform(X_test_withmissing[feature_name])
            x = X_test_encoded.iloc[:, feature].astype(float)
        else:
            x = X_test_withmissing.iloc[:, feature].astype(float)
    
    # Handle missing values in feature values and SHAP values
    mask_x = ~pd.isnull(x)
    mask_shap = ~np.isnan(shap_values[:, feature])
    mask = mask_x & mask_shap
    
    x_filtered = x[mask]
    shap_values_filtered = shap_values[:, feature][mask]
    predictions_filtered = predictions[mask]
    misclassified_filtered = misclassified[mask]
    
    # Check if all x values are identical
    if len(np.unique(x_filtered)) == 1:
        print(f"Skipped feature {feature_name} because all x values are identical.")
        continue
    
    # Calculate Spearman correlation coefficient and p-value
    correlation, p_value = spearmanr(x_filtered, shap_values_filtered, nan_policy='omit')
    
    # Create scatter plot in the current subplot
    scatter = axs[current_subplot].scatter(x_filtered, shap_values_filtered, c=predictions_filtered, cmap='viridis', alpha=0.7, s=50)
    axs[current_subplot].set_xlabel(feature_name)
    axs[current_subplot].set_ylabel("SHAP Value")
    
    # Add correlation line
    slope, intercept, r_value, p_value_corr, std_err = linregress(x_filtered, shap_values_filtered)
    axs[current_subplot].plot(x_filtered, slope * x_filtered + intercept, color='red')
    
    # Mark misclassified samples with 'x'
    axs[current_subplot].scatter(x_filtered[misclassified_filtered], shap_values_filtered[misclassified_filtered], marker="X", color='red', alpha=0.5, s=50)
    
    # Customize colorbar
    cbar = plt.colorbar(scatter, ax=axs[current_subplot])
    cbar.set_label("Predicted Probability")
    
    # Check if correlation is statistically significant
    if not np.isnan(correlation) and not np.isnan(p_value_corr):
        _, p_value_corr_test = ttest_rel(x_filtered, shap_values_filtered)
        p_value_text = f"p < 0.05" if p_value_corr_test < 0.05 else f"p = {p_value_corr_test:.2f}"
        axs[current_subplot].set_title(f"{feature_name} vs. SHAP Value\nSpearman Correlation: {correlation:.2f}, {p_value_text}")
    else:
        axs[current_subplot].set_title(f"{feature_name} vs. SHAP Value\nCorrelation: N/A")
    
    # Increment the current subplot index
    current_subplot += 1

# Hide any remaining empty subplots
for i in range(current_subplot, num_rows * num_cols):
    axs[i].axis('off')

# Adjust the spacing between subplots
plt.tight_layout()

# Show the plot
plt.show()

SHAP (SHapley Additive exPlanations) values are a method for explaining individual predictions in machine learning models. They provide insights into the contribution of each feature towards the predicted outcome for a specific instance.

The computation of SHAP values is based on cooperative game theory and the concept of Shapley values. SHAP values aim to fairly distribute the contribution of each feature across different subsets of features in the model. They provide a unified and consistent framework for feature importance estimation.

Interpreting SHAP plots can be done in the following way:

Feature Importance: The features are displayed on the y-axis, ordered from top to bottom based on their importance. The importance is determined by the magnitude of the SHAP values. Features at the top have the highest impact on the model's predictions.

Impact Direction: Each feature is represented by a horizontal bar. The color of the bar indicates the value of the feature for a specific instance. Red indicates high feature values, while blue represents low values.

SHAP Value: The length of the bar represents the magnitude of the SHAP value. Longer bars indicate higher contributions to increasing the prediction value, while shorter bars indicate lower contributions or even decreasing the prediction value.

Interaction Effects: If multiple features are correlated or interact with each other, the plot may show interactions. The overlapping of bars indicates that the features are dependent, and their combined effect differs from their individual effects.

Reference Point: The vertical gray dashed line represents the base value or the average prediction of the model. Features that push the prediction above this line contribute positively to the outcome, while features that pull it below the line contribute negatively.

By analyzing the SHAP plots, you can identify which features have the most significant impact on predictions and understand the direction and magnitude of their contributions. This information helps in gaining insights into the model's behavior and the relationship between features and predictions.

In [ ]:
from sklearn.metrics import roc_auc_score, matthews_corrcoef
import pandas as pd
import matplotlib.pyplot as plt
from scipy.stats import pearsonr  

# Convert the 'date' column to datetime
testset_backup['date'] = pd.to_datetime(testset_backup['date'])

# Initialize empty lists to store AUC and MCC values for each year
auc_values = []
mcc_values = []

# Loop through each year bin
for year_bin in range(2010, 2021):
    # Filter data for the current year bin
    test_data = testset_backup[testset_backup['date'].dt.year == year_bin]
    y_test_year_bin = y_test[y_test.index.isin(test_data.index)]

    if isinstance(selected_model, (HistGradientBoostingClassifier, BalancedRandomForestClassifier, LogisticRegression, GaussianNB)):
        X_test_year_bin = X_test_OHE[X_test_OHE.index.isin(test_data.index)]
        predictions_year_bin = selected_model.predict_proba(X_test_year_bin)
        predictions_year_bin = predictions[:, 1]
        # print(predictions_class)
        y_pred_year_bin = [True if x >= 0.5 else False for x in predictions_year_bin]
        misclassified_year_bin = y_pred_year_bin != y_test_year_bin
    elif isinstance(selected_model, (cb.CatBoostClassifier)):
        X_test_year_bin = X_test_withmissing[X_test_withmissing.index.isin(test_data.index)]
        predictions_year_bin = selected_model.predict_proba(X_test_year_bin)
        predictions_year_bin = predictions_year_bin[:, 1]
        # print(predictions_class)
        y_pred_year_bin = [True if x >= 0.5 else False for x in predictions_year_bin]
        misclassified_year_bin = y_pred_year_bin != y_test_year_bin
    else:
        X_test_year_bin = X_test_withmissing[X_test_withmissing.index.isin(test_data.index)]
        predictions_year_bin = selected_model.predict_proba(X_test_year_bin)
        predictions_year_bin = predictions_year_bin[:, 1]
        # print(predictions_class)
        y_pred_year_bin = [True if x >= 0.5 else False for x in predictions_year_bin]
        misclassified_year_bin = y_pred_year_bin != y_test_year_bin

    # Calculate AUC and MCC
    auc = roc_auc_score(y_test_year_bin, predictions_year_bin)
    mcc = matthews_corrcoef(y_test_year_bin, y_pred_year_bin)

    # Append values to the lists
    auc_values.append(auc)
    mcc_values.append(mcc)

years = range(2010, 2021)
# Perform Pearson correlation test between AUC and years
correlation_coefficient, p_value = pearsonr(auc_values, years)

# Print the correlation coefficient and p-value
print(f"Pearson Correlation Coefficient (AUC): {correlation_coefficient}")
print(f"P-value: {p_value}")

# Plotting AUC and MCC over the years
plt.plot(years, auc_values, label='AUC')
plt.plot(years, mcc_values, label='MCC')
plt.xlabel('Year')
plt.ylabel('Metric Value')
plt.title('AUC and MCC Over the Years')
plt.legend()
plt.show()



In [ ]:
import pandas as pd

test_index = y_test.index

# Identify correct and incorrect predictions when y_test is True
correctly_predicted_age_and_s = testset_backup.loc[test_index[(y_test == y_pred)], ['Age', 'Sex']]
incorrectly_predicted_age_and_s = testset_backup.loc[test_index[(y_test != y_pred)], ['Age', 'Sex']]

# Create DataFrames for correct and incorrect predictions
correct_results_df = pd.DataFrame({"Age": correctly_predicted_age_and_s['Age'], 'Sex': correctly_predicted_age_and_s['Sex']})
incorrect_results_df = pd.DataFrame({"Age": incorrectly_predicted_age_and_s['Age'], 'Sex': incorrectly_predicted_age_and_s['Sex']})

# Save DataFrames to separate Excel files
correct_results_df.to_excel("correctly_predicted_age_and_s.xlsx", index=False)
incorrect_results_df.to_excel("incorrectly_predicted_age_and_s.xlsx", index=False)

print("Results saved to correct_predictions.xlsx and incorrect_predictions.xlsx")


In [ ]:
from scipy.stats import mannwhitneyu
# Combine correct and incorrect predictions in a single DataFrame for plotting
combined_df = pd.concat([correct_results_df.assign(Type='Correct'), incorrect_results_df.assign(Type='Incorrect')])

# Plotting boxplot for correct and incorrect predictions
plt.figure(figsize=(5, 6))

sns.boxplot(x='Sex', y='Age', data=combined_df, hue='Type', palette='coolwarm')
plt.title('Distribution of age for correct and incorrect predictions')

# Statistical test (Mann-Whitney U test)
statistic, p_value = mannwhitneyu(correct_results_df['Age'], incorrect_results_df['Age'])
print(f"Mann-Whitney U test p-value: {p_value}")

plt.show()

In [ ]:

# Identify correct and incorrect predictions when y_test is True
correctly_predicted_ids = testset_backup.loc[test_index[(y_test == y_pred) & y_test], 'allbac']
incorrectly_predicted_ids = testset_backup.loc[test_index[(y_test != y_pred) & y_test], 'allbac']



# Create DataFrames for correct and incorrect predictions
correct_results_df = pd.DataFrame({"Correctly Predicted allbacs": correctly_predicted_ids})
incorrect_results_df = pd.DataFrame({"Incorrectly Predicted allbacs": incorrectly_predicted_ids})

# Save DataFrames to separate Excel files
correct_results_df.to_excel("correct_predictions.xlsx", index=False)
incorrect_results_df.to_excel("incorrect_predictions.xlsx", index=False)

print("Results saved to correct_predictions.xlsx and incorrect_predictions.xlsx")


In [ ]:
correctly_predicted_ids

In [ ]:
incorrectly_predicted_ids

In [ ]:
# Split names in the 'names' column and create a list of all names
all_names = [name.split(';') for name in correct_results_df['Correctly Predicted allbacs']]

# Flatten the list of lists
flat_list = [name for sublist in all_names for name in sublist]

# Create a Pandas Series from the flattened list
names_series = pd.Series(flat_list)

# Get the counts of each name
name_counts = names_series.value_counts()

# Plot the most frequent names
top_names = name_counts.head(10) 
top_names.plot(kind='bar', xlabel='Names', ylabel='Frequency', title='Top 10 most frequent pathogens correctly predicted')
plt.show()

In [ ]:
# Split names in the 'names' column and create a list of all names
all_names = [name.split(';') for name in correct_results_df['Correctly Predicted allbacs']]

# Flatten the list of lists
flat_list = [name for sublist in all_names for name in sublist]

# Create a Pandas Series from the flattened list
names_series = pd.Series(flat_list)

# Get the counts of each name
name_counts = names_series.value_counts()

# Plot the most frequent names
top_names = name_counts.head(10)  
top_names.plot(kind='bar', xlabel='Names', ylabel='Frequency', title='Top 10 most frequent pathogens correctly predicted')
plt.show()

# Split names in the 'names' column and create a list of all names
all_names = [name.split(';') for name in incorrect_results_df['Incorrectly Predicted allbacs']]

# Flatten the list of lists
flat_list = [name for sublist in all_names for name in sublist]

# Create a Pandas Series from the flattened list
names_series = pd.Series(flat_list)

# Get the counts of each name
name_counts = names_series.value_counts()

# Plot the most frequent names
top_names = name_counts.head(10)  
top_names.plot(kind='bar', xlabel='Names', ylabel='Frequency', title='Top 10 most frequent pathogens incorrectly predicted')
plt.show()

In [ ]:
import pandas as pd

# Extract names from correct predictions
all_names_correct = [name.split(';') for name in correct_results_df['Correctly Predicted allbacs']]
flat_list_correct = [name for sublist in all_names_correct for name in sublist]
names_series_correct = pd.Series(flat_list_correct)
name_counts_correct = names_series_correct.value_counts()

# Extract names from incorrect predictions
all_names_incorrect = [name.split(';') for name in incorrect_results_df['Incorrectly Predicted allbacs']]
flat_list_incorrect = [name for sublist in all_names_incorrect for name in sublist]
names_series_incorrect = pd.Series(flat_list_incorrect)
name_counts_incorrect = names_series_incorrect.value_counts()

# Find common names
common_names = list(set(name_counts_correct.index) & set(name_counts_incorrect.index))

# Create a DataFrame with counts for each common name
common_names_df = pd.DataFrame(index=common_names, columns=['Correct Counts', 'Incorrect Counts'])

# Fill in the counts for correct predictions
common_names_df['Correct Counts'] = name_counts_correct[common_names]

# Fill in the counts for incorrect predictions
common_names_df['Incorrect Counts'] = name_counts_incorrect[common_names]

# Display the resulting DataFrame
print(common_names_df)
common_names_df.to_excel("common_names_df.xlsx", index=True)

In [ ]:
# Create a DataFrame with counts for each common name
common_names_df = pd.DataFrame(index=common_names, columns=['Correctly classified samples', 'Incorrectly classified samples'])

# Fill in the counts for correct predictions
common_names_df['Correctly classified samples'] = name_counts_correct[common_names]

# Fill in the counts for incorrect predictions
common_names_df['Incorrectly classified samples'] = name_counts_incorrect[common_names]

# Sort by clusters (you might want to adjust the clustering method and metric)
clustered_df = common_names_df.copy()
clustered_df['Total Counts'] = clustered_df.sum(axis=1)
clustered_df = clustered_df.sort_values(by='Total Counts', ascending=False)
clustered_df = clustered_df.drop('Total Counts', axis=1)

# Set smaller font size
sns.set(font_scale=0.6)

# Create a heatmap
plt.figure(figsize=(8, 10))
sns.heatmap(clustered_df,annot=True,cmap='YlGnBu',fmt='g', cbar_kws={'label': 'Counts'})
plt.title('classification results by pathogens')
plt.savefig("classification_by_pathogens", dpi=300)
plt.show()

In [ ]:
import pandas as pd

df = clustered_df.copy()
# Calculate True Positive Rate (TPR) and add a new column
df['TruePositiveRate'] = df['Correctly classified samples'] / (df['Correctly classified samples'] + df['Incorrectly classified samples'])

# Sort the DataFrame by 'TruePositiveRate' in descending order
df = df.sort_values(by='TruePositiveRate', ascending=False)

# Plotting the True Positive Rate
fig, ax = plt.subplots(figsize=(10, 6))
df['TruePositiveRate'].plot(kind='bar', ax=ax, color='lightcoral', edgecolor='black')

# Customize background color
fig.patch.set_facecolor('white')
ax.set_facecolor('white')

# Adding labels and title
ax.set_xlabel('pathogens')
ax.set_ylabel('true positive rate')
ax.set_title('breakdown of true positive rates for pathogens in the test set')
plt.savefig("TPR_by_pathogens", dpi=300)
# Display the plot
plt.show()

In [ ]:
testset.shape

In [ ]:
len(devset_backup["ID"].unique())

In [ ]:
len(testset_backup["ID"].unique())